# 🎯 Conviction LEAPS — Trading Readiness Book

**Purpose:** You already have your names. This book deep-dives them for **real trade execution** — fundamentals, technicals, full IV analysis, Greeks, theta decay, IV rank timing, slippage, earnings risk, exit rules, and a pre-trade checklist.

**How to use:**

1. Edit `CONVICTION_TICKERS` in Cell 3 with your names
2. Set `STARTING_BALANCE` to your account size
3. Run All — get a full trading readiness report

**Pipeline:**

1. 📊 Fundamentals + Valuation
2. 📈 Technicals — HV, Sharpe, Sortino, Calmar, Bounce
3. 🎯 IV Deep Dive — Chains, IV Smile, Term Structure
4. 🏆 Scoring + Portfolio Construction
5. 📐 Greeks — Δ, Γ, Θ, V + Theta Decay Projection
6. 📊 IV Rank / Percentile — Entry Timing
7. 💰 Spread Slippage — Real Fill Cost
8. 📅 Earnings + Exit Rules + Stress Test + Pre-Trade Checklist


In [9]:
# ══════════════════════════════════════════════════════════════════
#  ⚙️  CONFIG + IMPORTS + SHARED FUNCTIONS
# ══════════════════════════════════════════════════════════════════

import time, warnings, math
from datetime import datetime, date
from typing import Optional

import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import norm

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from IPython.display import display, Markdown

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 60)
pd.set_option("display.width", 200)
pd.set_option("display.max_rows", 200)
pio.renderers.default = "notebook_connected"

# ── Constants ──
RISK_FREE_RATE = 0.045
RATE_LIMIT_SLEEP = 0.35
LEAPS_MIN_DTE = 300
LEAPS_MAX_DTE = 540
LEAPS_TARGET_DTE = 365
MED_DTE_MIN = 30
MED_DTE_MAX = 90
SHORT_DTE_MAX = 30
MONEYNESS_RANGE = (0.80, 1.05)
MED_MONEYNESS = (0.90, 1.05)
MIN_OPEN_INTEREST = 50
FRACTIONAL_KELLY = 0.25
KELLY_CAP = 0.18
EDGE_HAIRCUT = 0.90

STYLE_PREFS = {
    "preferred_moneyness_leaps": (0.55, 1.00),
    "preferred_moneyness_swing": (0.85, 1.00),
    "ideal_moneyness_leaps": 0.78,
    "ideal_breakeven_pct": 0.045,
    "max_var_per_contract": 55_450,
    "ideal_leverage": 8.3,
}

BUCKET_TARGET = {
    "⚡ <30 DTE": 0.10,
    "🔄 30-90 DTE": 0.25,
    "🏗️ 300+ DTE": 0.65,
}


# ══════════════════════════════════════════════════════════════════
#  SHARED FUNCTIONS
# ══════════════════════════════════════════════════════════════════


def safe_float(v, default=0):
    try:
        return (
            float(v)
            if v is not None and not (isinstance(v, float) and np.isnan(v))
            else default
        )
    except (TypeError, ValueError):
        return default


# ── BSM Greeks ──
def bsm_d1(S, K, T, r, sigma):
    if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
        return np.nan
    return (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))


def bsm_d2(S, K, T, r, sigma):
    d1 = bsm_d1(S, K, T, r, sigma)
    return np.nan if np.isnan(d1) else d1 - sigma * math.sqrt(T)


def bsm_call_price(S, K, T, r, sigma):
    d1 = bsm_d1(S, K, T, r, sigma)
    d2 = bsm_d2(S, K, T, r, sigma)
    if np.isnan(d1):
        return np.nan
    return S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)


def bsm_prob_profit(spot, breakeven, iv, dte, r=None):
    if r is None:
        r = RISK_FREE_RATE
    if iv <= 0 or dte <= 0 or spot <= 0 or breakeven <= 0:
        return np.nan
    T = dte / 365.0
    d2 = (math.log(spot / breakeven) + (r - 0.5 * iv**2) * T) / (iv * math.sqrt(T))
    return float(norm.cdf(d2))


def bsm_delta(S, K, T, r, sigma):
    d1 = bsm_d1(S, K, T, r, sigma)
    return float(norm.cdf(d1)) if not np.isnan(d1) else np.nan


def bsm_gamma(S, K, T, r, sigma):
    d1 = bsm_d1(S, K, T, r, sigma)
    if np.isnan(d1) or S <= 0 or sigma <= 0 or T <= 0:
        return np.nan
    return float(norm.pdf(d1) / (S * sigma * math.sqrt(T)))


def bsm_theta(S, K, T, r, sigma):
    d1 = bsm_d1(S, K, T, r, sigma)
    d2 = bsm_d2(S, K, T, r, sigma)
    if np.isnan(d1):
        return np.nan
    term1 = -(S * norm.pdf(d1) * sigma) / (2 * math.sqrt(T))
    term2 = -r * K * math.exp(-r * T) * norm.cdf(d2)
    return float((term1 + term2) / 365)


def bsm_vega(S, K, T, r, sigma):
    d1 = bsm_d1(S, K, T, r, sigma)
    if np.isnan(d1) or T <= 0:
        return np.nan
    return float(S * norm.pdf(d1) * math.sqrt(T) / 100)


# ── Scoring Functions ──
def score_option(row, hv_data, style_prefs):
    scores, weights = {}, {}
    ticker = row["ticker"]
    hv = hv_data.get(ticker, {})
    if not hv:
        return np.nan, np.nan, {}
    spot, iv, dte = row["spot"], row.get("iv", np.nan), row["dte"]
    breakeven, moneyness = row["breakeven"], row["moneyness"]

    p_profit = bsm_prob_profit(spot, breakeven, iv, dte) if not np.isnan(iv) else np.nan
    if not np.isnan(p_profit):
        scores["p_profit"] = min(p_profit * 100, 100)
        weights["p_profit"] = 0.25

    ret_1m = hv.get("ret_1m", 0)
    ret_3m = hv.get("ret_3m", 0)
    rsi = hv.get("rsi", 50)
    mom_score = 0
    mom_score += np.clip(ret_1m * 200, -30, 40)
    mom_score += np.clip(ret_3m * 100, -20, 30)
    if 45 < rsi < 72:
        mom_score += 15
    elif rsi >= 72:
        mom_score += 5
    if hv.get("above_50ma"):
        mom_score += 5
    if hv.get("above_200ma"):
        mom_score += 5
    if hv.get("golden_cross"):
        mom_score += 5
    scores["momentum"] = np.clip(mom_score, 0, 100)
    weights["momentum"] = 0.20

    be_pct = row.get("breakeven_pct", np.nan)
    if not np.isnan(be_pct):
        scores["breakeven"] = np.clip((0.15 - be_pct) / 0.20 * 100, 0, 100)
        weights["breakeven"] = 0.15

    hv_30 = hv.get("hv_30", np.nan)
    if not np.isnan(iv) and not np.isnan(hv_30) and iv > 0:
        iv_hv_ratio = iv / hv_30 if hv_30 > 0 else 2.0
        if iv_hv_ratio < 0.9:
            iv_score = 90
        elif iv_hv_ratio < 1.1:
            iv_score = 70
        elif iv_hv_ratio < 1.3:
            iv_score = 45
        else:
            iv_score = max(0, 30 - (iv_hv_ratio - 1.3) * 50)
        scores["iv_edge"] = iv_score
        weights["iv_edge"] = 0.15

    ideal_mon = style_prefs.get("ideal_moneyness_leaps", 0.78)
    mon_dist = abs(moneyness - ideal_mon)
    style_score = max(0, 100 - mon_dist * 500)
    if moneyness < 0.90:
        style_score = min(100, style_score + 10)
    scores["style_match"] = style_score
    weights["style_match"] = 0.15

    oi = safe_float(row.get("openInterest", 0))
    spread_pct = row.get("spread_pct", np.nan)
    liq_score = 0
    if oi > 500:
        liq_score += 40
    elif oi > 100:
        liq_score += 25
    elif oi > 20:
        liq_score += 10
    if not np.isnan(spread_pct):
        if spread_pct < 0.05:
            liq_score += 60
        elif spread_pct < 0.10:
            liq_score += 40
        elif spread_pct < 0.20:
            liq_score += 20
    scores["liquidity"] = min(liq_score, 100)
    weights["liquidity"] = 0.10

    total_w = sum(weights.values())
    if total_w == 0:
        return np.nan, p_profit, scores
    edge_score = sum(scores[k] * weights[k] for k in scores) / total_w
    return edge_score, p_profit, scores


def iv_enhanced_score(row, ratios, val_score):
    edge = row.get("Edge Score", 0)
    if np.isnan(edge):
        edge = 0
    bounce = row.get("Bounce Score", 0)
    if np.isnan(bounce):
        bounce = 0
    combo = edge * 0.65 + bounce * 0.35

    sharpe = ratios.get("sharpe", 0) or 0
    sortino = ratios.get("sortino", 0) or 0
    calmar = ratios.get("calmar", 0) or 0
    rr = row.get("R/R (+10%)", 0)
    if np.isnan(rr):
        rr = 0
    rr20 = row.get("R/R (+20%)", 0)
    if np.isnan(rr20):
        rr20 = 0
    win = ratios.get("win_rate", 0.5) or 0.5
    p_profit = row.get("P(Profit)", 0.3)
    if np.isnan(p_profit):
        p_profit = 0.3
    vs = val_score

    sharpe_n = np.clip((sharpe + 1) * 33, 0, 100)
    sortino_n = np.clip((sortino + 1) * 25, 0, 100)
    calmar_n = np.clip(calmar * 25, 0, 100)
    rr_n = np.clip(rr * 20, 0, 100)
    rr20_n = np.clip(rr20 * 10, 0, 100)
    win_n = np.clip(win * 100 - 20, 0, 100)
    pp_n = np.clip(p_profit * 100, 0, 100)

    bucket = row["Bucket"]
    if bucket == "🏗️ 300+ DTE":
        return (
            combo * 0.20
            + sharpe_n * 0.18
            + sortino_n * 0.15
            + calmar_n * 0.10
            + pp_n * 0.12
            + rr20_n * 0.10
            + win_n * 0.05
            + vs * 0.10
        )
    elif bucket == "🔄 30-90 DTE":
        return (
            combo * 0.25
            + sharpe_n * 0.20
            + sortino_n * 0.15
            + rr_n * 0.15
            + pp_n * 0.10
            + win_n * 0.10
            + vs * 0.05
        )
    else:
        return (
            combo * 0.30
            + rr_n * 0.25
            + sharpe_n * 0.15
            + win_n * 0.15
            + pp_n * 0.10
            + vs * 0.05
        )


def kelly_fraction(
    p_profit,
    reward_to_risk,
    fractional=FRACTIONAL_KELLY,
    cap=KELLY_CAP,
    haircut=EDGE_HAIRCUT,
):
    if np.isnan(p_profit) or np.isnan(reward_to_risk) or reward_to_risk <= 0:
        return 0.02
    p = p_profit * haircut
    q = 1 - p
    b = reward_to_risk
    f = (p * b - q) / b
    return float(np.clip(f * fractional, 0.02, cap))


def valuation_score(f):
    """Score fundamentals dict 0-100."""
    if not f:
        return 50
    score, n = 0, 0
    fwd_pe = f.get("fwd_pe")
    if fwd_pe and fwd_pe > 0:
        if fwd_pe < 12:
            score += 85
        elif fwd_pe < 18:
            score += 75
        elif fwd_pe < 25:
            score += 60
        elif fwd_pe < 35:
            score += 40
        elif fwd_pe < 50:
            score += 25
        else:
            score += 10
        n += 1
    peg = f.get("peg")
    if peg and peg > 0:
        if peg < 0.8:
            score += 90
        elif peg < 1.2:
            score += 75
        elif peg < 2.0:
            score += 50
        elif peg < 3.0:
            score += 30
        else:
            score += 10
        n += 1
    fcf_y = f.get("fcf_yield")
    if fcf_y is not None:
        if fcf_y > 6:
            score += 90
        elif fcf_y > 4:
            score += 75
        elif fcf_y > 2:
            score += 55
        elif fcf_y > 0:
            score += 35
        else:
            score += 10
        n += 1
    rg = f.get("rev_growth")
    if rg is not None:
        if rg > 0.20:
            score += 85
        elif rg > 0.10:
            score += 65
        elif rg > 0.05:
            score += 50
        elif rg > 0:
            score += 35
        else:
            score += 15
        n += 1
    pm_val = f.get("profit_margin")
    if pm_val is not None:
        if pm_val > 0.30:
            score += 85
        elif pm_val > 0.20:
            score += 70
        elif pm_val > 0.10:
            score += 50
        elif pm_val > 0:
            score += 30
        else:
            score += 10
        n += 1
    return score / n if n > 0 else 50


print("✅ Config loaded — all functions ready")

✅ Config loaded — all functions ready


In [10]:
# ══════════════════════════════════════════════════════════════════
#  🎯  INPUT — Edit These Tickers, Then Run All Cells
# ══════════════════════════════════════════════════════════════════

CONVICTION_TICKERS = ["GEV", "WDC", "KLAC"]
COMPARE_TICKERS = ["TSM", "GOOG", "MRVL", "AMAT"]  # contextual benchmarks

STARTING_BALANCE = 15_000
today = date.today()

display(Markdown("# 🎯 Conviction LEAPS — Trading Readiness"))
display(
    Markdown(
        f"**Date:** {today}  |  **Capital:** ${STARTING_BALANCE:,}  |  **Tickers:** {', '.join(CONVICTION_TICKERS)}"
    )
)

# ── Fetch basic info for all tickers ──
print("Fetching ticker info…\n")

ticker_info = {}
spot_map = {}
sector_map = {}
name_map = {}
beta_map = {}
fundamentals = {}

for ticker in CONVICTION_TICKERS + COMPARE_TICKERS:
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        info = t.info
        ticker_info[ticker] = info

        price = info.get("currentPrice") or info.get("previousClose", 0)
        if price <= 0:
            h = t.history(period="5d")
            price = float(h["Close"].iloc[-1]) if not h.empty else 0

        spot_map[ticker] = price
        sector_map[ticker] = info.get("sector", "—")
        name_map[ticker] = info.get("shortName", ticker)
        beta_map[ticker] = info.get("beta", 1.0)

        # Populate fundamentals dict
        fwd_pe = info.get("forwardPE") or info.get("forwardPe")
        trail_pe = info.get("trailingPE") or info.get("trailingPe")
        peg_v = info.get("pegRatio") or info.get("trailingPegRatio")
        mcap_v = info.get("marketCap", 0)
        fcf_v = info.get("freeCashflow", 0)
        ev_v = info.get("enterpriseValue", 0)
        total_debt = info.get("totalDebt", 0)
        total_cash = info.get("totalCash", 0)
        ebitda_v = info.get("ebitda", 0)
        fcf_yield = (fcf_v / mcap_v * 100) if (mcap_v and fcf_v) else None
        net_debt = (total_debt - total_cash) if (total_debt and total_cash) else None
        nd_ebitda = (
            (net_debt / ebitda_v)
            if (net_debt is not None and ebitda_v and ebitda_v > 0)
            else None
        )

        fundamentals[ticker] = {
            "fwd_pe": fwd_pe,
            "trail_pe": trail_pe,
            "peg": peg_v,
            "mcap": mcap_v,
            "fcf_yield": fcf_yield,
            "rev_growth": info.get("revenueGrowth"),
            "profit_margin": info.get("profitMargins"),
            "gross_margin": info.get("grossMargins"),
            "op_margin": info.get("operatingMargins"),
            "roe": info.get("returnOnEquity"),
            "debt_to_equity": info.get("debtToEquity"),
            "ps": info.get("priceToSalesTrailing12Months"),
            "nd_ebitda": nd_ebitda,
        }
        tag = "🎯" if ticker in CONVICTION_TICKERS else "📊"
        print(f"  {tag} {ticker:5s}: ${price:>9,.2f}  {sector_map[ticker]}")
    except Exception as e:
        print(f"  ⚠ {ticker}: {e}")

print(f"\n✅ Loaded {len(spot_map)} tickers")

# 🎯 Conviction LEAPS — Trading Readiness

**Date:** 2026-02-08  |  **Capital:** $15,000  |  **Tickers:** GEV, WDC, KLAC

Fetching ticker info…

  🎯 GEV  : $   779.35  Industrials
  🎯 WDC  : $   282.58  Technology
  🎯 KLAC : $ 1,442.95  Technology
  📊 TSM  : $   348.85  Technology
  📊 GOOG : $   323.10  Communication Services
  📊 MRVL : $    80.28  Technology
  📊 AMAT : $   322.51  Technology

✅ Loaded 7 tickers


In [11]:
# ══════════════════════════════════════════════════════════════════
#  📊  STEP 1: Fundamentals + Historical Performance & Risk Ratios
# ══════════════════════════════════════════════════════════════════

display(Markdown("---\n## 📊 Fundamentals Comparison"))

fund_rows = []
val_scores = {}
for ticker in CONVICTION_TICKERS + COMPARE_TICKERS:
    info = ticker_info.get(ticker, {})
    if not info:
        continue
    vs = valuation_score(fundamentals.get(ticker, {}))
    val_scores[ticker] = vs
    fund_rows.append(
        {
            "Ticker": ticker,
            "Name": name_map.get(ticker, ticker),
            "Sector": sector_map.get(ticker, "—"),
            "Price": spot_map.get(ticker, 0),
            "MCap ($B)": info.get("marketCap", 0) / 1e9,
            "PE": info.get("trailingPE") or info.get("forwardPE"),
            "PEG": info.get("pegRatio"),
            "ROE": info.get("returnOnEquity"),
            "Rev Growth": info.get("revenueGrowth"),
            "Gross Margin": info.get("grossMargins"),
            "Op Margin": info.get("operatingMargins"),
            "Profit Margin": info.get("profitMargins"),
            "β": info.get("beta", 1.0),
            "D/E": info.get("debtToEquity"),
            "Inst %": info.get("heldPercentInstitutions"),
            "FCF ($M)": info.get("freeCashflow", 0) / 1e6
            if info.get("freeCashflow")
            else 0,
            "Val Score": vs,
            "Focus": "🎯" if ticker in CONVICTION_TICKERS else "📊",
        }
    )

fund_df = pd.DataFrame(fund_rows)
display(
    fund_df.style.format(
        {
            "Price": "${:,.2f}",
            "MCap ($B)": "{:,.1f}",
            "PE": "{:.1f}",
            "PEG": "{:.2f}",
            "ROE": "{:.1%}",
            "Rev Growth": "{:.1%}",
            "Gross Margin": "{:.1%}",
            "Op Margin": "{:.1%}",
            "Profit Margin": "{:.1%}",
            "β": "{:.2f}",
            "D/E": "{:.0f}",
            "Inst %": "{:.0%}",
            "FCF ($M)": "{:,.0f}",
            "Val Score": "{:.0f}",
        },
        na_rep="—",
    )
    .background_gradient(subset=["Val Score"], cmap="RdYlGn", vmin=20, vmax=80)
    .background_gradient(subset=["ROE"], cmap="RdYlGn", vmin=0, vmax=0.4)
    .set_caption("Fundamentals: Focus (🎯) vs Comparison (📊)")
)


# ══════════════════════════════════════════════════════════════════
#  📈  Historical Performance & Risk Ratios
# ══════════════════════════════════════════════════════════════════

display(Markdown("---\n## 📈 Historical Performance & Risk Ratios"))

ticker_ratios = {}
hv_momentum = {}
bounce_rows = []
daily_rf = RISK_FREE_RATE / 252

for ticker in CONVICTION_TICKERS + COMPARE_TICKERS:
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        hist = t.history(period="1y")
        if hist.empty or len(hist) < 60:
            continue
        closes = hist["Close"]
        daily_ret = closes.pct_change().dropna()

        ann_ret = (1 + daily_ret.mean()) ** 252 - 1
        ann_vol = daily_ret.std() * np.sqrt(252)

        log_ret = np.log(closes / closes.shift(1)).dropna()
        hv_30 = log_ret.tail(30).std() * np.sqrt(252) if len(log_ret) >= 30 else ann_vol
        hv_60 = log_ret.tail(60).std() * np.sqrt(252) if len(log_ret) >= 60 else ann_vol
        hv_90 = log_ret.tail(90).std() * np.sqrt(252) if len(log_ret) >= 90 else ann_vol

        excess = daily_ret - daily_rf
        sharpe = excess.mean() / excess.std() * np.sqrt(252) if excess.std() > 0 else 0
        down = excess[excess < 0]
        down_vol = down.std() * np.sqrt(252) if len(down) > 5 else ann_vol
        sortino = (ann_ret - RISK_FREE_RATE) / down_vol if down_vol > 0 else 0
        cum = (1 + daily_ret).cumprod()
        peak = cum.cummax()
        dd = (cum - peak) / peak
        max_dd = dd.min()
        calmar = (ann_ret - RISK_FREE_RATE) / abs(max_dd) if max_dd < 0 else 0
        win_rate = (daily_ret > 0).mean()

        ret_1m = (closes.iloc[-1] / closes.iloc[-22] - 1) if len(closes) >= 22 else 0
        ret_3m = (closes.iloc[-1] / closes.iloc[-66] - 1) if len(closes) >= 66 else 0
        ret_6m = (closes.iloc[-1] / closes.iloc[-126] - 1) if len(closes) >= 126 else 0
        ret_1y = closes.iloc[-1] / closes.iloc[0] - 1

        # RSI
        delta_p = closes.diff()
        gain_s = delta_p.clip(lower=0).rolling(14).mean()
        loss_s = (-delta_p.clip(upper=0)).rolling(14).mean()
        rs_v = (gain_s / loss_s).iloc[-1] if loss_s.iloc[-1] > 0 else 100
        rsi = 100 - (100 / (1 + rs_v))

        # MAs
        ma50 = (
            closes.rolling(50).mean().iloc[-1] if len(closes) >= 50 else closes.iloc[-1]
        )
        ma200 = (
            closes.rolling(200).mean().iloc[-1]
            if len(closes) >= 200
            else closes.iloc[-1]
        )
        above_50 = closes.iloc[-1] > ma50
        above_200 = closes.iloc[-1] > ma200 if len(closes) >= 200 else False
        golden_cross = ma50 > ma200 if len(closes) >= 200 else False

        ticker_ratios[ticker] = {
            "ann_ret": ann_ret,
            "ann_vol": ann_vol,
            "hv_30": hv_30,
            "hv_60": hv_60,
            "hv_90": hv_90,
            "sharpe": sharpe,
            "sortino": sortino,
            "calmar": calmar,
            "max_dd": max_dd,
            "win_rate": win_rate,
            "ret_1m": ret_1m,
            "ret_3m": ret_3m,
            "ret_6m": ret_6m,
            "ret_1y": ret_1y,
        }
        hv_momentum[ticker] = {
            "hv_30": hv_30,
            "hv_60": hv_60,
            "hv_90": hv_90,
            "ret_1m": ret_1m,
            "ret_3m": ret_3m,
            "ret_6m": ret_6m,
            "rsi": rsi,
            "above_50ma": above_50,
            "above_200ma": above_200,
            "golden_cross": golden_cross,
        }

        # Bounce scanner
        ma20 = (
            closes.rolling(20).mean().iloc[-1] if len(closes) >= 20 else closes.iloc[-1]
        )
        bb_std = closes.rolling(20).std()
        bb_lower = closes.rolling(20).mean() - 2 * bb_std
        bb_pct_b = (
            (closes.iloc[-1] - bb_lower.iloc[-1]) / (4 * bb_std.iloc[-1])
            if bb_std.iloc[-1] > 0
            else 0.5
        )
        dist_ma50 = (closes.iloc[-1] - ma50) / ma50 * 100
        dist_ma200 = (
            (closes.iloc[-1] - ma200) / ma200 * 100 if len(closes) >= 200 else 0
        )
        high_52w = closes.max()
        low_52w = closes.min()
        dist_52w = (closes.iloc[-1] - high_52w) / high_52w * 100

        # Bounce scoring
        b_score = 0
        if bb_pct_b < 0.2:
            b_score += 25
        if dist_ma50 < -5:
            b_score += 20
        if dist_52w < -15:
            b_score += 20
        stoch_k_s = (
            (closes - closes.rolling(14).min())
            / (closes.rolling(14).max() - closes.rolling(14).min())
        ) * 100
        stoch_k = stoch_k_s.iloc[-1] if not stoch_k_s.empty else 50
        stoch_d = (
            stoch_k_s.rolling(3).mean().iloc[-1] if len(stoch_k_s) >= 3 else stoch_k
        )
        if stoch_k < 20:
            b_score += 15
        macd_line = closes.ewm(span=12).mean() - closes.ewm(span=26).mean()
        macd_signal = macd_line.ewm(span=9).mean()
        macd_hist = macd_line - macd_signal
        macd_turning = (
            len(macd_hist) >= 2
            and float(macd_hist.iloc[-1]) > float(macd_hist.iloc[-2])
            and float(macd_hist.iloc[-1]) < 0
        )
        if macd_turning:
            b_score += 10
        if rsi < 35:
            b_score += 10

        bounce_rows.append(
            {
                "Ticker": ticker,
                "Bounce Score": b_score,
                "BB %B": bb_pct_b,
                "RSI": rsi,
                "Stoch %K": stoch_k,
                "Dist MA50": dist_ma50,
                "Dist 52W High": dist_52w,
            }
        )
    except Exception as e:
        print(f"  ⚠ {ticker}: {e}")

bounce_df = pd.DataFrame(bounce_rows)

# ── Display ratio table ──
perf_rows = []
for ticker in CONVICTION_TICKERS + COMPARE_TICKERS:
    r = ticker_ratios.get(ticker, {})
    if not r:
        continue
    perf_rows.append(
        {
            "Ticker": ticker,
            "Ann Ret": r["ann_ret"],
            "Ann Vol": r["ann_vol"],
            "HV30": r["hv_30"],
            "HV60": r["hv_60"],
            "HV90": r["hv_90"],
            "Sharpe": r["sharpe"],
            "Sortino": r["sortino"],
            "Calmar": r["calmar"],
            "Max DD": r["max_dd"],
            "Win Rate": r["win_rate"],
            "1M": r["ret_1m"],
            "3M": r["ret_3m"],
            "6M": r["ret_6m"],
            "1Y": r["ret_1y"],
            "Focus": "🎯" if ticker in CONVICTION_TICKERS else "📊",
        }
    )

perf_df = pd.DataFrame(perf_rows)
display(
    perf_df.style.format(
        {
            "Ann Ret": "{:+.1%}",
            "Ann Vol": "{:.1%}",
            "HV30": "{:.1%}",
            "HV60": "{:.1%}",
            "HV90": "{:.1%}",
            "Sharpe": "{:+.2f}",
            "Sortino": "{:+.2f}",
            "Calmar": "{:+.2f}",
            "Max DD": "{:.1%}",
            "Win Rate": "{:.1%}",
            "1M": "{:+.1%}",
            "3M": "{:+.1%}",
            "6M": "{:+.1%}",
            "1Y": "{:+.1%}",
        },
        na_rep="—",
    )
    .background_gradient(subset=["Sharpe"], cmap="RdYlGn", vmin=-0.5, vmax=3.0)
    .background_gradient(subset=["Sortino"], cmap="RdYlGn", vmin=-0.5, vmax=4.0)
    .set_caption("Performance & Risk — Focus (🎯) vs Comparison (📊)")
)

# ── Charts ──
fig_perf = go.Figure()
for metric, color in [
    ("Sharpe", "#3498db"),
    ("Sortino", "#2ecc71"),
    ("Calmar", "#e67e22"),
]:
    fig_perf.add_trace(
        go.Bar(
            name=metric,
            x=perf_df["Ticker"],
            y=perf_df[metric],
            marker_color=color,
            text=[f"{v:+.2f}" for v in perf_df[metric]],
            textposition="outside",
        )
    )
fig_perf.add_hline(y=0, line_dash="dot", line_color="gray")
fig_perf.update_layout(
    barmode="group", title="Risk-Adjusted Ratios", height=450, yaxis_title="Ratio"
)
fig_perf.show()

fig_hv = go.Figure()
for window, color in [("HV30", "#e74c3c"), ("HV60", "#f39c12"), ("HV90", "#3498db")]:
    fig_hv.add_trace(
        go.Bar(
            name=window,
            x=perf_df["Ticker"],
            y=perf_df[window],
            marker_color=color,
            text=[f"{v:.0%}" for v in perf_df[window]],
            textposition="outside",
        )
    )
fig_hv.update_layout(
    barmode="group",
    title="Historical Volatility (30/60/90d)",
    height=400,
    yaxis_title="Annualised HV",
    yaxis_tickformat=".0%",
)
fig_hv.show()

print(f"\n✅ Performance data for {len(ticker_ratios)} tickers")

---
## 📊 Fundamentals Comparison

,Ticker,Name,Sector,Price,MCap ($B),PE,PEG,ROE,Rev Growth,Gross Margin,Op Margin,Profit Margin,β,D/E,Inst %,FCF ($M),Val Score,Focus
0,GEV,GE Vernova Inc.,Industrials,$779.35,211.5,44.1,—,42.6%,3.8%,20.1%,7.4%,12.8%,1.00,10,80%,"5,278",35,🎯
1,WDC,Western Digital Corporation,Technology,$282.58,96.6,26.7,—,41.1%,—,42.7%,15.4%,35.6%,1.84,—,107%,"3,899",74,🎯
2,KLAC,KLA Corporation,Technology,"$1,442.95",189.6,42.0,—,100.7%,7.2%,61.6%,41.3%,35.8%,1.46,112,94%,"3,223",48,🎯
3,TSM,Taiwan Semiconductor Manufactur,Technology,$348.85,"1,809.3",33.3,—,35.2%,20.5%,59.9%,53.8%,45.1%,1.27,18,16%,"619,090",74,📊
4,GOOG,Alphabet Inc.,Communication Services,$323.10,"3,908.5",29.9,—,35.7%,18.0%,59.7%,31.6%,32.8%,1.09,16,60%,"38,088",55,📊
5,MRVL,"Marvell Technology, Inc.",Technology,$80.28,69.2,28.3,—,18.0%,36.8%,50.7%,17.7%,31.7%,1.98,34,82%,"1,954",71,📊
6,AMAT,"Applied Materials, Inc.",Technology,$322.51,256.0,37.2,—,35.5%,-3.5%,48.7%,28.4%,24.7%,1.68,35,85%,"3,653",38,📊


---
## 📈 Historical Performance & Risk Ratios

,Ticker,Ann Ret,Ann Vol,HV30,HV60,HV90,Sharpe,Sortino,Calmar,Max DD,Win Rate,1M,3M,6M,1Y,Focus
0,GEV,+139.5%,53.7%,42.7%,58.9%,54.1%,+1.55,+3.84,+4.52,-29.8%,54.8%,+17.7%,+34.2%,+20.2%,+106.7%,🎯
1,WDC,+617.7%,63.0%,94.8%,81.3%,77.5%,+3.07,+13.98,+13.97,-43.9%,61.6%,+41.4%,+79.0%,+277.7%,+482.2%,🎯
2,KLAC,+119.8%,48.4%,73.7%,58.3%,54.5%,+1.54,+3.14,+4.49,-25.7%,58.8%,+6.1%,+18.6%,+58.3%,+94.6%,🎯
3,TSM,+85.3%,38.4%,36.5%,33.3%,36.5%,+1.49,+3.32,+2.52,-32.0%,53.6%,+9.5%,+14.7%,+45.1%,+71.5%,📊
4,GOOG,+82.3%,30.6%,19.6%,26.4%,26.8%,+1.82,+4.16,+3.53,-22.0%,53.6%,+0.2%,+13.8%,+60.1%,+73.3%,📊
5,MRVL,-9.9%,65.2%,41.2%,51.8%,51.7%,-0.23,-0.28,-0.26,-56.2%,49.2%,-5.1%,-11.1%,+3.9%,-27.2%,📊
6,AMAT,+103.4%,47.5%,53.4%,47.6%,47.0%,+1.40,+2.82,+3.20,-30.9%,55.2%,+10.4%,+35.9%,+75.3%,+81.0%,📊



✅ Performance data for 7 tickers


In [12]:
# ══════════════════════════════════════════════════════════════════
#  🔬  STEP 2: IV Deep Dive — Full LEAPS Chains + IV Smile + Term Structure
# ══════════════════════════════════════════════════════════════════

display(Markdown("---\n## 🔬 IV Deep Dive — LEAPS Chains"))
for t_name, p in spot_map.items():
    if t_name in CONVICTION_TICKERS:
        print(f"  {t_name}: ${p:,.2f}  ({sector_map.get(t_name, '—')})")

print("\nFetching option chains…\n")

focus_chains = []
iv_by_ticker = {}

for ticker in CONVICTION_TICKERS:
    spot = spot_map.get(ticker, 0)
    if spot <= 0:
        continue
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        exps = t.options
        if not exps:
            print(f"  ⚠ {ticker}: no expirations")
            continue

        ticker_iv_data = {}
        leaps_exps_f, med_exps_f, short_exps_f = [], [], []
        for exp_str in exps:
            try:
                exp_date = datetime.strptime(exp_str, "%Y-%m-%d").date()
                dte = (exp_date - today).days
                if dte <= 0:
                    continue
                if LEAPS_MIN_DTE <= dte <= LEAPS_MAX_DTE:
                    leaps_exps_f.append((exp_str, dte))
                elif MED_DTE_MIN <= dte <= MED_DTE_MAX:
                    med_exps_f.append((exp_str, dte))
                elif 5 <= dte < SHORT_DTE_MAX:
                    short_exps_f.append((exp_str, dte))
            except ValueError:
                continue

        fetched = 0
        for bucket_type, exp_list, mon_range in [
            ("LEAPS", leaps_exps_f, (0.55, 1.10)),
            ("MED", med_exps_f, (0.85, 1.10)),
            ("SHORT", short_exps_f, (0.90, 1.05)),
        ]:
            for exp_str, dte in exp_list[:3]:
                try:
                    chain = t.option_chain(exp_str)
                    time.sleep(RATE_LIMIT_SLEEP)
                    calls = chain.calls
                    if calls.empty:
                        continue
                    iv_points = []
                    for _, opt in calls.iterrows():
                        strike = safe_float(opt.get("strike"))
                        if strike <= 0:
                            continue
                        moneyness = strike / spot
                        if not (mon_range[0] <= moneyness <= mon_range[1]):
                            continue
                        bid = safe_float(opt.get("bid"), 0)
                        ask = safe_float(opt.get("ask"), 0)
                        mid = (
                            (bid + ask) / 2
                            if bid > 0 and ask > 0
                            else safe_float(opt.get("lastPrice"), 0)
                        )
                        if mid <= 0:
                            continue
                        iv = safe_float(opt.get("impliedVolatility"), np.nan)
                        oi_val = safe_float(opt.get("openInterest"), 0)
                        spread_pct = (
                            (ask - bid) / mid
                            if mid > 0 and bid > 0 and ask > 0
                            else np.nan
                        )
                        breakeven = strike + mid
                        breakeven_pct = (breakeven - spot) / spot
                        leverage = spot / mid if mid > 0 else 0

                        focus_chains.append(
                            {
                                "ticker": ticker,
                                "expiration": exp_str,
                                "dte": dte,
                                "strike": strike,
                                "spot": spot,
                                "moneyness": moneyness,
                                "bid": bid,
                                "ask": ask,
                                "mid": mid,
                                "iv": iv if iv > 0 else np.nan,
                                "openInterest": oi_val,
                                "spread_pct": spread_pct,
                                "breakeven": breakeven,
                                "breakeven_pct": breakeven_pct,
                                "leverage": leverage,
                                "cost_per_contract": mid * 100,
                                "bucket_type": bucket_type,
                            }
                        )
                        if not np.isnan(iv) and iv > 0:
                            iv_points.append((moneyness, iv))
                        fetched += 1
                    if iv_points:
                        ticker_iv_data[exp_str] = iv_points
                except Exception:
                    continue

        iv_by_ticker[ticker] = ticker_iv_data
        print(
            f"  {ticker}: {fetched} options ({len(leaps_exps_f)} LEAPS, {len(med_exps_f)} MED, {len(short_exps_f)} SHORT exps)"
        )
    except Exception as e:
        print(f"  ⚠ {ticker}: {e}")

focus_chain_df = pd.DataFrame(focus_chains)
print(f"\n✅ {len(focus_chain_df)} options loaded")


# ── ATM IV Analysis ──
display(Markdown("---\n### 📊 ATM IV by Ticker & Expiry"))

atm_iv_rows = []
for ticker in CONVICTION_TICKERS:
    for exp_str, iv_points in iv_by_ticker.get(ticker, {}).items():
        if not iv_points:
            continue
        sorted_pts = sorted(iv_points, key=lambda x: abs(x[0] - 1.0))
        atm_iv = sorted_pts[0][1] if sorted_pts else np.nan
        deep_pts = sorted(iv_points, key=lambda x: abs(x[0] - 0.80))
        deep_iv = (
            deep_pts[0][1] if deep_pts and abs(deep_pts[0][0] - 0.80) < 0.10 else np.nan
        )
        skew = atm_iv - deep_iv if not np.isnan(deep_iv) else np.nan
        exp_date = datetime.strptime(exp_str, "%Y-%m-%d").date()
        dte = (exp_date - today).days
        hv30 = hv_momentum.get(ticker, {}).get("hv_30", np.nan)
        atm_iv_rows.append(
            {
                "Ticker": ticker,
                "Expiry": exp_str,
                "DTE": dte,
                "ATM IV": atm_iv,
                "Deep ITM IV (80%)": deep_iv,
                "Skew": skew,
                "HV30": hv30,
                "IV / HV": atm_iv / hv30
                if hv30 and hv30 > 0 and not np.isnan(atm_iv)
                else np.nan,
            }
        )

atm_df = pd.DataFrame(atm_iv_rows)
if not atm_df.empty:
    display(
        atm_df.style.format(
            {
                "ATM IV": "{:.1%}",
                "Deep ITM IV (80%)": "{:.1%}",
                "Skew": "{:+.1%}",
                "HV30": "{:.1%}",
                "IV / HV": "{:.2f}x",
            },
            na_rep="—",
        )
        .background_gradient(subset=["ATM IV"], cmap="YlOrRd", vmin=0.20, vmax=0.60)
        .background_gradient(subset=["IV / HV"], cmap="RdYlGn_r", vmin=0.7, vmax=1.5)
        .set_caption("IV Analysis — IV/HV > 1.2 = overpriced")
    )


# ── IV Smile ──
display(Markdown("### 😊 IV Smile"))
fig_iv = make_subplots(
    rows=1,
    cols=len(CONVICTION_TICKERS),
    subplot_titles=CONVICTION_TICKERS,
    shared_yaxes=True,
)
colors_palette = ["#e74c3c", "#3498db", "#2ecc71", "#f39c12", "#9b59b6"]

for i, ticker in enumerate(CONVICTION_TICKERS, 1):
    exp_data = iv_by_ticker.get(ticker, {})
    for j, (exp_str, iv_points) in enumerate(exp_data.items()):
        if not iv_points:
            continue
        sorted_pts = sorted(iv_points, key=lambda x: x[0])
        xs = [p[0] for p in sorted_pts]
        ys = [p[1] for p in sorted_pts]
        fig_iv.add_trace(
            go.Scatter(
                x=xs,
                y=ys,
                mode="lines+markers",
                name=f"{exp_str}",
                marker=dict(color=colors_palette[j % len(colors_palette)], size=5),
                showlegend=(i == 1),
            ),
            row=1,
            col=i,
        )
    fig_iv.add_vline(x=1.0, line_dash="dot", line_color="gray", row=1, col=i)

fig_iv.update_layout(
    title="IV Smile by Expiry", height=400, yaxis_title="IV", yaxis_tickformat=".0%"
)
for i in range(1, len(CONVICTION_TICKERS) + 1):
    fig_iv.update_xaxes(tickformat=".0%", title_text="Moneyness", row=1, col=i)
fig_iv.show()


# ── IV Term Structure ──
display(Markdown("### 📐 IV Term Structure (ATM by DTE)"))
fig_ts = go.Figure()
for ticker in CONVICTION_TICKERS:
    sub = atm_df[atm_df["Ticker"] == ticker].sort_values("DTE")
    if not sub.empty:
        fig_ts.add_trace(
            go.Scatter(
                x=sub["DTE"],
                y=sub["ATM IV"],
                mode="lines+markers",
                name=ticker,
                text=sub["Expiry"],
                hovertemplate=f"{ticker}<br>DTE: %{{x}}<br>IV: %{{y:.1%}}<br>Exp: %{{text}}",
            )
        )
fig_ts.update_layout(
    title="ATM IV Term Structure",
    xaxis_title="Days to Expiry",
    yaxis_title="ATM IV",
    yaxis_tickformat=".0%",
    height=400,
)
fig_ts.show()

---
## 🔬 IV Deep Dive — LEAPS Chains

  GEV: $779.35  (Industrials)
  WDC: $282.58  (Technology)
  KLAC: $1,442.95  (Technology)

Fetching option chains…

  GEV: 201 options (2 LEAPS, 3 MED, 4 SHORT exps)
  WDC: 106 options (2 LEAPS, 3 MED, 4 SHORT exps)
  KLAC: 156 options (3 LEAPS, 2 MED, 1 SHORT exps)

✅ 463 options loaded


---
### 📊 ATM IV by Ticker & Expiry

,Ticker,Expiry,DTE,ATM IV,Deep ITM IV (80%),Skew,HV30,IV / HV
0,GEV,2026-12-18,313,55.1%,59.6%,-4.5%,42.7%,1.29x
1,GEV,2027-01-15,341,55.6%,58.8%,-3.2%,42.7%,1.30x
2,GEV,2026-03-13,33,50.8%,54.0%,-3.2%,42.7%,1.19x
3,GEV,2026-03-20,40,50.7%,56.2%,-5.5%,42.7%,1.19x
4,GEV,2026-04-17,68,50.5%,55.1%,-4.6%,42.7%,1.18x
5,GEV,2026-02-13,5,53.5%,—,—,42.7%,1.25x
6,GEV,2026-02-20,12,49.6%,—,—,42.7%,1.16x
7,GEV,2026-02-27,19,51.3%,—,—,42.7%,1.20x
8,WDC,2026-12-18,313,82.4%,84.3%,-1.9%,94.8%,0.87x
9,WDC,2027-01-15,341,83.5%,84.4%,-0.9%,94.8%,0.88x


### 😊 IV Smile

### 📐 IV Term Structure (ATM by DTE)

In [13]:
# ══════════════════════════════════════════════════════════════════
#  🏆  STEP 3: IV-Enhanced Scoring + Concentrated Portfolio
# ══════════════════════════════════════════════════════════════════

display(Markdown("---\n## 🏆 IV-Enhanced Scoring"))

focus_scored = []
for _, row in focus_chain_df.iterrows():
    ticker = row["ticker"]
    hv_data = hv_momentum.get(ticker, {})
    if not hv_data:
        continue
    edge, p_profit, components = score_option(row, {ticker: hv_data}, STYLE_PREFS)
    if np.isnan(edge):
        continue

    dte = row["dte"]
    bucket = (
        "🏗️ 300+ DTE" if dte >= 300 else ("🔄 30-90 DTE" if dte >= 30 else "⚡ <30 DTE")
    )
    b_match = bounce_df[bounce_df["Ticker"] == ticker]
    b_score = float(b_match["Bounce Score"].iloc[0]) if not b_match.empty else 0

    upside_spot = row["spot"] * 1.10
    payout = max(upside_spot - row["strike"], 0)
    rr = (payout - row["mid"]) / row["mid"] if row["mid"] > 0 else 0
    upside_20 = row["spot"] * 1.20
    payout_20 = max(upside_20 - row["strike"], 0)
    rr_20 = (payout_20 - row["mid"]) / row["mid"] if row["mid"] > 0 else 0
    intrinsic = max(0, row["spot"] - row["strike"])
    extrinsic = row["mid"] - intrinsic
    extrinsic_pct = extrinsic / row["mid"] if row["mid"] > 0 else 0
    vs = val_scores.get(ticker, 50)

    focus_scored.append(
        {
            "Ticker": ticker,
            "Name": name_map.get(ticker, ticker),
            "Sector": sector_map.get(ticker, "—"),
            "Bucket": bucket,
            "Expiry": row["expiration"],
            "DTE": dte,
            "Strike": row["strike"],
            "Moneyness": row["moneyness"],
            "Spot": row["spot"],
            "Mid": row["mid"],
            "IV": row.get("iv", np.nan),
            "HV30": hv_data.get("hv_30", np.nan),
            "IV/HV": row.get("iv", np.nan) / hv_data.get("hv_30", 1.0)
            if not np.isnan(row.get("iv", np.nan))
            else np.nan,
            "Breakeven %": row["breakeven_pct"],
            "Leverage": row["leverage"],
            "Cost/Contract": row["cost_per_contract"],
            "OI": safe_float(row.get("openInterest", 0)),
            "Spread %": row.get("spread_pct", np.nan),
            "P(Profit)": p_profit,
            "Edge Score": edge,
            "Bounce Score": b_score,
            "R/R (+10%)": rr,
            "R/R (+20%)": rr_20,
            "Extrinsic %": extrinsic_pct,
            "Val Score": vs,
            "s_pprofit": components.get("p_profit", np.nan),
            "s_momentum": components.get("momentum", np.nan),
            "s_breakeven": components.get("breakeven", np.nan),
            "s_iv_edge": components.get("iv_edge", np.nan),
            "s_style": components.get("style_match", np.nan),
            "s_liquidity": components.get("liquidity", np.nan),
        }
    )

focus_scored_df = pd.DataFrame(focus_scored)

# Attach risk ratios
for ticker in CONVICTION_TICKERS:
    r = ticker_ratios.get(ticker, {})
    mask = focus_scored_df["Ticker"] == ticker
    focus_scored_df.loc[mask, "Sharpe"] = r.get("sharpe", 0)
    focus_scored_df.loc[mask, "Sortino"] = r.get("sortino", 0)
    focus_scored_df.loc[mask, "Calmar"] = r.get("calmar", 0)
    focus_scored_df.loc[mask, "Max DD"] = r.get("max_dd", 0)
    focus_scored_df.loc[mask, "Win Rate"] = r.get("win_rate", 0.5)
    focus_scored_df.loc[mask, "Ann Ret"] = r.get("ann_ret", 0)
    focus_scored_df.loc[mask, "β"] = beta_map.get(ticker, 1.0)

# IV-enhanced score (uses the function from config cell)
focus_scored_df["IV Score"] = focus_scored_df.apply(
    lambda row: iv_enhanced_score(
        row,
        ticker_ratios.get(row["Ticker"], {}),
        val_scores.get(row["Ticker"], 50),
    ),
    axis=1,
)

print(
    f"✅ Scored {len(focus_scored_df)} options — range {focus_scored_df['IV Score'].min():.0f}–{focus_scored_df['IV Score'].max():.0f}"
)


# ── Best LEAPS per ticker ──
display(Markdown("### 🏗️ Best LEAPS"))
for ticker in CONVICTION_TICKERS:
    sub = focus_scored_df[
        (focus_scored_df["Ticker"] == ticker)
        & (focus_scored_df["Bucket"] == "🏗️ 300+ DTE")
    ].copy()
    if sub.empty:
        display(Markdown(f"\n**{ticker}**: No LEAPS available"))
        continue
    sub = sub.sort_values("IV Score", ascending=False).head(8)
    r = ticker_ratios.get(ticker, {})
    display(Markdown(f"\n### {ticker} — {name_map.get(ticker, '')}"))
    display(
        Markdown(
            f"Sector: **{sector_map.get(ticker, '—')}** | Spot: **${spot_map.get(ticker, 0):,.2f}** | "
            f"β: **{beta_map.get(ticker, 1.0):.2f}** | Sharpe: **{r.get('sharpe', 0):+.2f}** | "
            f"Sortino: **{r.get('sortino', 0):+.2f}** | Max DD: **{r.get('max_dd', 0):.1%}**"
        )
    )
    dc = [
        c
        for c in [
            "Expiry",
            "DTE",
            "Strike",
            "Moneyness",
            "Mid",
            "IV",
            "HV30",
            "IV/HV",
            "Breakeven %",
            "Leverage",
            "P(Profit)",
            "R/R (+10%)",
            "R/R (+20%)",
            "Edge Score",
            "IV Score",
        ]
        if c in sub.columns
    ]
    display(
        sub[dc]
        .style.format(
            {
                "Strike": "${:,.0f}",
                "Mid": "${:,.2f}",
                "Moneyness": "{:.0%}",
                "IV": "{:.1%}",
                "HV30": "{:.1%}",
                "IV/HV": "{:.2f}x",
                "Breakeven %": "{:+.1%}",
                "Leverage": "{:.0f}x",
                "P(Profit)": "{:.0%}",
                "R/R (+10%)": "{:.1f}x",
                "R/R (+20%)": "{:.2f}x",
                "Edge Score": "{:.0f}",
                "IV Score": "{:.0f}",
            },
            na_rep="—",
        )
        .background_gradient(subset=["IV Score"], cmap="RdYlGn", vmin=30, vmax=70)
        .set_caption(f"{ticker} — Top LEAPS")
    )


# ── Radar chart ──
display(Markdown("### 🕸️ Ticker Comparison Radar"))
radar_cats = [
    "Sharpe",
    "Sortino",
    "Calmar",
    "Val Score",
    "Best Edge",
    "HV (IV Potential)",
]
fig_radar = go.Figure()
for ticker in CONVICTION_TICKERS:
    r = ticker_ratios.get(ticker, {})
    best_edge = focus_scored_df[focus_scored_df["Ticker"] == ticker]["Edge Score"].max()
    vals = [
        np.clip((r.get("sharpe", 0) + 1) * 33, 0, 100),
        np.clip((r.get("sortino", 0) + 1) * 25, 0, 100),
        np.clip(r.get("calmar", 0) * 25, 0, 100),
        val_scores.get(ticker, 50),
        best_edge if not np.isnan(best_edge) else 0,
        np.clip(r.get("hv_30", 0.30) * 200, 0, 100),
    ]
    fig_radar.add_trace(
        go.Scatterpolar(
            r=vals + [vals[0]],
            theta=radar_cats + [radar_cats[0]],
            fill="toself",
            name=ticker,
            opacity=0.6,
        )
    )
fig_radar.update_layout(
    polar=dict(radialaxis=dict(visible=True, range=[0, 100])),
    title="Conviction Ticker Comparison",
    height=500,
)
fig_radar.show()


# ══════════════════════════════════════════════════════════════════
#  📦  CONCENTRATED PORTFOLIO
# ══════════════════════════════════════════════════════════════════

display(Markdown("---\n## 📦 Concentrated Portfolio"))
display(
    Markdown(
        f"**Capital:** ${STARTING_BALANCE:,.0f} | **Names:** {', '.join(CONVICTION_TICKERS)} | **Strategy:** LEAPS-heavy"
    )
)

conc_pool = focus_scored_df.copy()
conc_pool["Cost/1"] = conc_pool["Mid"] * 100
PER_TICKER_TARGET = STARTING_BALANCE / len(CONVICTION_TICKERS)
conc_positions = []
conc_remaining = float(STARTING_BALANCE)

print(f"🔄 Building portfolio — ${PER_TICKER_TARGET:,.0f}/name…\n")

for ticker in CONVICTION_TICKERS:
    ticker_spent = 0
    ticker_pool = conc_pool[conc_pool["Ticker"] == ticker].sort_values(
        "IV Score", ascending=False
    )
    for bucket_pref in ["🏗️ 300+ DTE", "🔄 30-90 DTE", "⚡ <30 DTE"]:
        if ticker_spent >= PER_TICKER_TARGET * 0.90:
            break
        bucket_opts = ticker_pool[ticker_pool["Bucket"] == bucket_pref].copy()
        if bucket_opts.empty:
            continue
        ticker_budget = min(PER_TICKER_TARGET - ticker_spent, conc_remaining)
        affordable = bucket_opts[bucket_opts["Cost/1"] <= ticker_budget]
        if affordable.empty:
            continue
        best = affordable.iloc[0]
        cost_1 = best["Mid"] * 100
        p_profit = best.get("P(Profit)", 0.5)
        rr_val = best.get("R/R (+10%)", 1.0)
        if np.isnan(rr_val):
            rr_val = 1.0
        kf = kelly_fraction(p_profit, max(rr_val, 0.01))
        kelly_amt = STARTING_BALANCE * max(kf, 0.03)
        alloc = min(kelly_amt, ticker_budget, conc_remaining)
        contracts = max(1, int(alloc / cost_1))
        total_cost = contracts * cost_1
        if total_cost > conc_remaining:
            contracts = max(1, int(conc_remaining / cost_1))
            total_cost = contracts * cost_1
        if total_cost > conc_remaining or total_cost < 50:
            continue

        conc_positions.append(
            {
                "Ticker": ticker,
                "Name": best.get("Name", ticker),
                "Sector": best.get("Sector", "—"),
                "Bucket": bucket_pref,
                "Expiry": best.get("Expiry", "—"),
                "DTE": best.get("DTE", 0),
                "Strike": best.get("Strike", 0),
                "Moneyness": best.get("Moneyness", 1.0),
                "Mid": best["Mid"],
                "IV": best.get("IV", np.nan),
                "Breakeven %": best.get("Breakeven %", np.nan),
                "Leverage": best.get("Leverage", 1),
                "P(Profit)": p_profit,
                "Sharpe": best.get("Sharpe", 0),
                "Sortino": best.get("Sortino", 0),
                "R/R (+10%)": rr_val,
                "R/R (+20%)": best.get("R/R (+20%)", 0),
                "Edge Score": best.get("Edge Score", 0),
                "IV Score": best.get("IV Score", 0),
                "Val Score": best.get("Val Score", 50),
                "Kelly %": kf,
                "Contracts": contracts,
                "Total Cost": total_cost,
            }
        )
        conc_remaining -= total_cost
        ticker_spent += total_cost
        print(
            f"  ✅ {bucket_pref} | {ticker:5s} | {contracts}× ${best['Strike']:,.0f} {best.get('Expiry', '—')} | IV {best.get('IV', 0):.0%} | ${total_cost:,.0f}"
        )

conc_df = pd.DataFrame(conc_positions)
conc_deployed = conc_df["Total Cost"].sum() if not conc_df.empty else 0
conc_cash = STARTING_BALANCE - conc_deployed
conc_pct = conc_deployed / STARTING_BALANCE * 100

print(f"\n{'=' * 55}")
print(f"💰 Deployed: ${conc_deployed:,.0f} ({conc_pct:.0f}%)")
print(f"💵 Cash: ${conc_cash:,.0f} ({100 - conc_pct:.0f}%)")
print(
    f"📊 Positions: {len(conc_df)} across {conc_df['Ticker'].nunique() if not conc_df.empty else 0} tickers"
)
print(f"{'=' * 55}")


# ══════════════════════════════════════════════════════════════════
#  📊  PORTFOLIO DASHBOARD
# ══════════════════════════════════════════════════════════════════
if not conc_df.empty:
    display(Markdown("---\n## 📊 Portfolio Dashboard"))

    # Holdings table
    h_cols = [
        "Ticker",
        "Sector",
        "Bucket",
        "Expiry",
        "DTE",
        "Strike",
        "Moneyness",
        "Mid",
        "Contracts",
        "Total Cost",
        "IV",
        "P(Profit)",
        "Leverage",
        "Sharpe",
        "Sortino",
        "R/R (+10%)",
        "R/R (+20%)",
        "Edge Score",
        "IV Score",
        "Val Score",
        "Kelly %",
    ]
    hc = [c for c in h_cols if c in conc_df.columns]
    display(
        conc_df[hc]
        .style.format(
            {
                "Strike": "${:,.0f}",
                "Mid": "${:,.2f}",
                "Moneyness": "{:.0%}",
                "Total Cost": "${:,.0f}",
                "IV": "{:.1%}",
                "P(Profit)": "{:.0%}",
                "Leverage": "{:.0f}x",
                "Sharpe": "{:+.2f}",
                "Sortino": "{:+.2f}",
                "R/R (+10%)": "{:.1f}x",
                "R/R (+20%)": "{:.2f}x",
                "Edge Score": "{:.0f}",
                "IV Score": "{:.0f}",
                "Val Score": "{:.0f}",
                "Kelly %": "{:.1%}",
            },
            na_rep="—",
        )
        .background_gradient(subset=["IV Score"], cmap="RdYlGn", vmin=25, vmax=65)
        .set_caption(f"Portfolio: {len(conc_df)} positions, ${conc_deployed:,.0f}")
    )

    # Risk summary
    w = conc_df["Total Cost"] / conc_deployed
    wp = (conc_df["P(Profit)"] * w).sum()
    wrr = (conc_df["R/R (+10%)"] * w).sum()
    ws = (conc_df["Sharpe"] * w).sum()
    wso = (conc_df["Sortino"] * w).sum()
    wdte = (conc_df["DTE"] * w).sum()
    wlev = (conc_df["Leverage"] * w).sum()
    wiv = conc_df["IV"].dropna()
    avg_iv = (wiv * w[wiv.index]).sum() if len(wiv) > 0 else 0
    hhi = (w**2).sum()

    # Pies
    fig_cp = make_subplots(
        rows=1,
        cols=3,
        specs=[[{"type": "pie"}] * 3],
        subplot_titles=["By Bucket", "By Ticker", "By Sector"],
    )
    ba = conc_df.groupby("Bucket")["Total Cost"].sum()
    fig_cp.add_trace(
        go.Pie(
            labels=ba.index.tolist(),
            values=ba.values,
            hole=0.4,
            textinfo="label+percent",
        ),
        1,
        1,
    )
    ta = conc_df.groupby("Ticker")["Total Cost"].sum()
    fig_cp.add_trace(
        go.Pie(
            labels=ta.index.tolist(),
            values=ta.values,
            hole=0.4,
            textinfo="label+percent",
        ),
        1,
        2,
    )
    sa = conc_df.groupby("Sector")["Total Cost"].sum()
    fig_cp.add_trace(
        go.Pie(
            labels=sa.index.tolist(),
            values=sa.values,
            hole=0.4,
            textinfo="label+percent",
        ),
        1,
        3,
    )
    fig_cp.update_layout(height=400, title_text="Portfolio Allocation")
    fig_cp.show()

    # Scenario P&L
    display(Markdown("### 📈 Scenario P&L"))
    scenarios = [-0.20, -0.15, -0.10, -0.05, 0.0, 0.05, 0.10, 0.15, 0.20, 0.30, 0.50]
    scen_pnl = {}
    for s in scenarios:
        total = 0
        for _, row in conc_df.iterrows():
            spot = spot_map.get(row["Ticker"], row["Strike"])
            new_spot = spot * (1 + s)
            intr = max(0, new_spot - row["Strike"]) * row["Contracts"] * 100
            total += intr - row["Total Cost"]
        scen_pnl[f"{s:+.0%}"] = total

    spnl = pd.DataFrame(list(scen_pnl.items()), columns=["Move", "P&L"])
    cols = ["#e74c3c" if v < 0 else "#2ecc71" for v in spnl["P&L"]]
    fig_spnl = go.Figure(
        go.Bar(
            x=spnl["Move"],
            y=spnl["P&L"],
            marker_color=cols,
            text=[f"${v:,.0f}" for v in spnl["P&L"]],
            textposition="outside",
        )
    )
    fig_spnl.add_hline(y=0, line_dash="dot", line_color="gray")
    fig_spnl.add_hline(
        y=-conc_deployed, line_dash="dash", line_color="red", annotation_text="Max Loss"
    )
    fig_spnl.update_layout(
        title="Scenario P&L at Expiry",
        xaxis_title="Stock Move",
        yaxis_title="P&L ($)",
        height=450,
    )
    fig_spnl.show()

    # Per-position returns
    display(Markdown("### 📊 Per-Position Returns"))
    ret_rows = []
    for _, row in conc_df.iterrows():
        spot = spot_map.get(row["Ticker"], row["Strike"])
        rr = {
            "Ticker": row["Ticker"],
            "Bucket": row["Bucket"],
            "Strike": row["Strike"],
            "Cost": row["Total Cost"],
        }
        for s in [-0.15, -0.10, -0.05, 0.0, 0.05, 0.10, 0.15, 0.20, 0.30, 0.50]:
            new_spot = spot * (1 + s)
            intr = max(0, new_spot - row["Strike"]) * row["Contracts"] * 100
            pnl = intr - row["Total Cost"]
            rr[f"{s:+.0%}"] = (
                pnl / row["Total Cost"] * 100 if row["Total Cost"] > 0 else 0
            )
        ret_rows.append(rr)
    rdf = pd.DataFrame(ret_rows)
    rc = [
        f"{s:+.0%}"
        for s in [-0.15, -0.10, -0.05, 0.0, 0.05, 0.10, 0.15, 0.20, 0.30, 0.50]
    ]
    display(
        rdf.style.format(
            {c: "{:+.0f}%" for c in rc} | {"Cost": "${:,.0f}", "Strike": "${:,.0f}"}
        )
        .background_gradient(subset=rc, cmap="RdYlGn", vmin=-100, vmax=150)
        .set_caption("Return (%) by scenario at expiry")
    )

    # Correlation heatmap
    focus_returns = {}
    for ticker in CONVICTION_TICKERS:
        try:
            t = yf.Ticker(ticker)
            time.sleep(RATE_LIMIT_SLEEP)
            h = t.history(period="6mo")
            if not h.empty and len(h) > 30:
                focus_returns[ticker] = h["Close"].pct_change().dropna()
        except Exception:
            pass
    if len(focus_returns) > 1:
        fc = pd.DataFrame(focus_returns).corr()
        display(Markdown("### 🔗 Correlation"))
        fig_fc = go.Figure(
            data=go.Heatmap(
                z=fc.values,
                x=fc.columns.tolist(),
                y=fc.index.tolist(),
                colorscale="RdYlGn_r",
                zmin=-0.2,
                zmax=1.0,
                text=np.round(fc.values, 2),
                texttemplate="%{text}",
                textfont={"size": 14},
            )
        )
        fig_fc.update_layout(
            title=f"6M Pairwise Correlation — {' · '.join(CONVICTION_TICKERS)}",
            height=350,
            width=400,
        )
        fig_fc.show()
        avg_pair_corr = fc.values[np.triu_indices_from(fc, k=1)].mean()
    else:
        avg_pair_corr = 0

else:
    display(Markdown("⚠️ No positions could be built."))
    avg_pair_corr = 0

---
## 🏆 IV-Enhanced Scoring

✅ Scored 463 options — range 36–72


### 🏗️ Best LEAPS


### GEV — GE Vernova Inc.

Sector: **Industrials** | Spot: **$779.35** | β: **1.00** | Sharpe: **+1.55** | Sortino: **+3.84** | Max DD: **-29.8%**

...

,Ticker,Sector,Bucket,Expiry,DTE,Strike,Moneyness,Mid,Contracts,Total Cost,IV,P(Profit),Leverage,Sharpe,Sortino,R/R (+10%),R/R (+20%),Edge Score,IV Score,Val Score,Kelly %
0,GEV,Industrials,🔄 30-90 DTE,2026-03-13,33,$785,101%,$45.55,1,"$4,555",50.8%,32%,17x,+1.55,+3.84,0.6x,2.30x,44,49,35,2.0%
1,WDC,Technology,🔄 30-90 DTE,2026-03-13,33,$250,88%,$47.62,1,"$4,762",88.0%,38%,6x,+3.07,+13.98,0.3x,0.87x,60,57,74,2.0%
2,KLAC,Technology,🔄 30-90 DTE,2026-03-20,40,"$1,580",109%,$46.10,1,"$4,610",51.5%,22%,31x,+1.54,+3.14,-0.8x,2.29x,33,47,48,2.0%


### 📈 Scenario P&L

### 📊 Per-Position Returns

,Ticker,Bucket,Strike,Cost,-15%,-10%,-5%,+0%,+5%,+10%,+15%,+20%,+30%,+50%
0,GEV,🔄 30-90 DTE,$785,"$4,555",-100%,-100%,-100%,-100%,-27%,+59%,+144%,+230%,+401%,+743%
1,WDC,🔄 30-90 DTE,$250,"$4,762",-100%,-91%,-61%,-32%,-2%,+28%,+57%,+87%,+146%,+265%
2,KLAC,🔄 30-90 DTE,"$1,580","$4,610",-100%,-100%,-100%,-100%,-100%,-84%,+72%,+229%,+542%,+1168%


### 🔗 Correlation

In [14]:
# ══════════════════════════════════════════════════════════════════
#  📐  STEP 4: Greeks + Theta Decay Projection
# ══════════════════════════════════════════════════════════════════

display(Markdown("---\n## 📐 Greeks — Delta · Gamma · Theta · Vega"))

greeks_rows = []
for _, row in conc_df.iterrows():
    ticker = row["Ticker"]
    S = spot_map.get(ticker, row["Strike"])
    K = row["Strike"]
    T = row["DTE"] / 365.0
    r = RISK_FREE_RATE
    sigma = row["IV"] if not np.isnan(row.get("IV", np.nan)) else 0.30

    delta = bsm_delta(S, K, T, r, sigma)
    gamma = bsm_gamma(S, K, T, r, sigma)
    theta = bsm_theta(S, K, T, r, sigma)
    vega = bsm_vega(S, K, T, r, sigma)
    contracts = row["Contracts"]
    cost = row["Total Cost"]

    delta_dollars = delta * S * contracts * 100 if not np.isnan(delta) else 0
    theta_daily = theta * contracts * 100 if not np.isnan(theta) else 0
    vega_dollars = vega * contracts * 100 if not np.isnan(vega) else 0
    theta_pct_day = (
        abs(theta_daily) / cost * 100 if cost > 0 and theta_daily != 0 else 0
    )
    days_to_50pct = (cost * 0.50) / abs(theta_daily) if theta_daily != 0 else np.inf
    days_to_25pct = (cost * 0.25) / abs(theta_daily) if theta_daily != 0 else np.inf
    bsm_price = bsm_call_price(S, K, T, r, sigma)
    theo_vs_market = (
        ((bsm_price - row["Mid"]) / row["Mid"] * 100)
        if bsm_price and row["Mid"] > 0
        else 0
    )

    greeks_rows.append(
        {
            "Ticker": ticker,
            "Bucket": row["Bucket"],
            "Strike": K,
            "DTE": row["DTE"],
            "Spot": S,
            "Mid": row["Mid"],
            "IV": sigma,
            "Contracts": contracts,
            "Cost": cost,
            "Δ (Delta)": delta,
            "Γ (Gamma)": gamma,
            "Θ (Theta)": theta,
            "V (Vega)": vega,
            "Δ$ (Dollar Delta)": delta_dollars,
            "Θ$/day": theta_daily,
            "V$ (per 1% IV)": vega_dollars,
            "Θ%/day": theta_pct_day,
            "Days to -25%": days_to_25pct,
            "Days to -50%": days_to_50pct,
            "BSM Price": bsm_price,
            "Theo vs Mkt": theo_vs_market,
        }
    )

greeks_df = pd.DataFrame(greeks_rows)

gcols = [
    "Ticker",
    "Strike",
    "DTE",
    "Contracts",
    "Cost",
    "Δ (Delta)",
    "Γ (Gamma)",
    "Θ (Theta)",
    "V (Vega)",
    "Δ$ (Dollar Delta)",
    "Θ$/day",
    "V$ (per 1% IV)",
]
display(
    greeks_df[gcols]
    .style.format(
        {
            "Strike": "${:,.0f}",
            "Cost": "${:,.0f}",
            "Δ (Delta)": "{:.3f}",
            "Γ (Gamma)": "{:.5f}",
            "Θ (Theta)": "{:.3f}",
            "V (Vega)": "{:.3f}",
            "Δ$ (Dollar Delta)": "${:+,.0f}",
            "Θ$/day": "${:,.2f}",
            "V$ (per 1% IV)": "${:+,.0f}",
        },
        na_rep="—",
    )
    .background_gradient(subset=["Δ (Delta)"], cmap="Blues", vmin=0.3, vmax=1.0)
    .background_gradient(subset=["Θ$/day"], cmap="Reds_r", vmin=-30, vmax=0)
    .set_caption("Position Greeks")
)

# Portfolio-level
total_delta_dollars = greeks_df["Δ$ (Dollar Delta)"].sum()
total_theta_daily = greeks_df["Θ$/day"].sum()
total_vega_dollars = greeks_df["V$ (per 1% IV)"].sum()
total_theta_abs = abs(total_theta_daily)

display(Markdown("### 🏦 Portfolio-Level Exposure"))
display(
    Markdown(f"""
| Metric | Value | Meaning |
|--------|-------|---------|
| **Total Δ$** | ${total_delta_dollars:+,.0f} | Effectively long **${abs(total_delta_dollars):,.0f}** of stock |
| **Total Θ$/day** | ${total_theta_daily:,.2f} | **${total_theta_abs:,.2f}/day** time decay |
| **Θ$/week** | ${total_theta_daily * 7:,.2f} | **${total_theta_abs * 7:,.2f}/week** bleed |
| **Θ$/month** | ${total_theta_daily * 30:,.2f} | **${total_theta_abs * 30:,.2f}/month** bleed |
| **Θ as % of portfolio/day** | {total_theta_abs / conc_deployed * 100:.3f}% | Daily "rent" on positions |
| **Total V$** | ${total_vega_dollars:+,.0f} | 1% IV move = **±${abs(total_vega_dollars):,.0f}** |
| **Deployed** | ${conc_deployed:,.0f} | {conc_pct:.0f}% of capital at risk |
""")
)


# ── Theta Decay Projection ──
display(Markdown("### 📉 Theta Decay Projection"))
decay_rows = []
for _, row in greeks_df.iterrows():
    ticker = row["Ticker"]
    S, K = row["Spot"], row["Strike"]
    r, sigma, contracts = RISK_FREE_RATE, row["IV"], row["Contracts"]
    dte_now, cost = row["DTE"], row["Cost"]
    for days_forward in range(0, dte_now + 1, max(1, dte_now // 40)):
        dte_remaining = dte_now - days_forward
        if dte_remaining <= 0:
            intrinsic = max(0, S - K) * contracts * 100
            decay_rows.append(
                {
                    "Ticker": ticker,
                    "Day": days_forward,
                    "DTE Remaining": 0,
                    "BSM Value": intrinsic,
                    "Value as % Cost": intrinsic / cost * 100 if cost > 0 else 0,
                }
            )
            break
        T = dte_remaining / 365.0
        bv = bsm_call_price(S, K, T, r, sigma)
        if bv is None or np.isnan(bv):
            continue
        pv = bv * contracts * 100
        decay_rows.append(
            {
                "Ticker": ticker,
                "Day": days_forward,
                "DTE Remaining": dte_remaining,
                "BSM Value": pv,
                "Value as % Cost": pv / cost * 100 if cost > 0 else 0,
            }
        )

decay_df = pd.DataFrame(decay_rows)

fig_decay = go.Figure()
cp = ["#e74c3c", "#3498db", "#2ecc71", "#f39c12", "#9b59b6"]
for i, ticker in enumerate(conc_df["Ticker"].unique()):
    td = decay_df[decay_df["Ticker"] == ticker]
    if td.empty:
        continue
    fig_decay.add_trace(
        go.Scatter(
            x=td["Day"],
            y=td["Value as % Cost"],
            mode="lines",
            name=f"{ticker} (DTE {int(conc_df[conc_df['Ticker'] == ticker]['DTE'].iloc[0])})",
            line=dict(color=cp[i % len(cp)], width=2),
        )
    )
fig_decay.add_hline(
    y=100, line_dash="dot", line_color="gray", annotation_text="Break-even"
)
fig_decay.add_hline(
    y=75, line_dash="dot", line_color="orange", annotation_text="−25% stop"
)
fig_decay.add_hline(
    y=50, line_dash="dot", line_color="red", annotation_text="−50% stop"
)
fig_decay.update_layout(
    title="Position Value Over Time (stock flat) — Pure Theta Decay",
    xaxis_title="Days Held",
    yaxis_title="Value (% of Cost)",
    height=500,
)
fig_decay.show()

# Decay milestones
decay_summary = []
for _, row in greeks_df.iterrows():
    td = decay_df[decay_df["Ticker"] == row["Ticker"]].sort_values("Day")
    day_75 = (
        td[td["Value as % Cost"] <= 75]["Day"].min()
        if len(td[td["Value as % Cost"] <= 75]) > 0
        else np.nan
    )
    day_50 = (
        td[td["Value as % Cost"] <= 50]["Day"].min()
        if len(td[td["Value as % Cost"] <= 50]) > 0
        else np.nan
    )
    decay_summary.append(
        {
            "Ticker": row["Ticker"],
            "DTE": row["DTE"],
            "Cost": row["Cost"],
            "Θ$/day": row["Θ$/day"],
            "Θ%/day": row["Θ%/day"],
            "Days to −25%": day_75,
            "Days to −50%": day_50,
            "Monthly Bleed $": abs(row["Θ$/day"]) * 30,
            "Monthly Bleed %": row["Θ%/day"] * 30,
        }
    )
display(
    pd.DataFrame(decay_summary)
    .style.format(
        {
            "Cost": "${:,.0f}",
            "Θ$/day": "${:,.2f}",
            "Θ%/day": "{:.2f}%",
            "Days to −25%": "{:.0f}",
            "Days to −50%": "{:.0f}",
            "Monthly Bleed $": "${:,.0f}",
            "Monthly Bleed %": "{:.1f}%",
        },
        na_rep="Beyond DTE",
    )
    .set_caption("Theta Decay Milestones (stock flat)")
)

print(
    f"\n💡 Total theta: ${total_theta_abs:,.2f}/day → ~${total_theta_abs * 30:,.0f}/month decay to overcome."
)

---
## 📐 Greeks — Delta · Gamma · Theta · Vega

,Ticker,Strike,DTE,Contracts,Cost,Δ (Delta),Γ (Gamma),Θ (Theta),V (Vega),Δ$ (Dollar Delta),Θ$/day,V$ (per 1% IV)
0,GEV,$785,33,1,"$4,555",0.522,0.00335,-0.763,0.933,"$+40,698",$-76.27,$+93
1,WDC,$250,33,1,"$4,762",0.729,0.00443,-0.395,0.281,"$+20,608",$-39.46,$+28
2,KLAC,"$1,580",40,1,"$4,610",0.338,0.00149,-1.178,1.746,"$+48,758",$-117.77,$+175


### 🏦 Portfolio-Level Exposure


| Metric | Value | Meaning |
|--------|-------|---------|
| **Total Δ$** | $+110,064 | Effectively long **$110,064** of stock |
| **Total Θ$/day** | $-233.50 | **$233.50/day** time decay |
| **Θ$/week** | $-1,634.49 | **$1,634.49/week** bleed |
| **Θ$/month** | $-7,004.94 | **$7,004.94/month** bleed |
| **Θ as % of portfolio/day** | 1.677% | Daily "rent" on positions |
| **Total V$** | $+296 | 1% IV move = **±$296** |
| **Deployed** | $13,928 | 93% of capital at risk |


### 📉 Theta Decay Projection

,Ticker,DTE,Cost,Θ$/day,Θ%/day,Days to −25%,Days to −50%,Monthly Bleed $,Monthly Bleed %
0,GEV,33,"$4,555",$-76.27,1.67%,15,24,"$2,288",50.2%
1,WDC,33,"$4,762",$-39.46,0.83%,26,Beyond DTE,"$1,184",24.9%
2,KLAC,40,"$4,610",$-117.77,2.55%,13,22,"$3,533",76.6%



💡 Total theta: $233.50/day → ~$7,005/month decay to overcome.


In [15]:
# ══════════════════════════════════════════════════════════════════
#  📊  STEP 5: IV Rank + Slippage Analysis
# ══════════════════════════════════════════════════════════════════

display(Markdown("---\n## 📊 IV Rank & IV Percentile — Entry Timing"))
display(
    Markdown(
        "> **IV Rank** = where current IV sits vs 52-week range (0 = cheapest, 100 = most expensive).\n"
        "> 🟢 Buy when IV Rank < 30.  🔴 Avoid buying when IV Rank > 70."
    )
)

iv_rank_rows = []
for ticker in CONVICTION_TICKERS:
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        hist = t.history(period="1y")
        if hist.empty or len(hist) < 60:
            continue
        closes = hist["Close"].dropna()
        log_ret = np.log(closes / closes.shift(1)).dropna()
        rolling_hv = (log_ret.rolling(30).std() * math.sqrt(252)).dropna()
        if len(rolling_hv) < 30:
            continue

        current_hv = float(rolling_hv.iloc[-1])
        hv_52w_high = float(rolling_hv.max())
        hv_52w_low = float(rolling_hv.min())
        hv_range = hv_52w_high - hv_52w_low
        iv_rank = ((current_hv - hv_52w_low) / hv_range * 100) if hv_range > 0 else 50
        iv_percentile = float((rolling_hv < current_hv).sum() / len(rolling_hv) * 100)

        atm_options = focus_chain_df[
            (focus_chain_df["ticker"] == ticker)
            & (focus_chain_df["moneyness"].between(0.95, 1.05))
            & (focus_chain_df["iv"] > 0)
        ]
        current_atm_iv = (
            float(atm_options["iv"].median()) if not atm_options.empty else np.nan
        )
        iv_hv_premium = (
            (current_atm_iv - current_hv) / current_hv * 100
            if current_hv > 0 and not np.isnan(current_atm_iv)
            else np.nan
        )

        if iv_rank < 25:
            signal = "🟢 IV LOW — good time to buy"
        elif iv_rank < 40:
            signal = "🟡 IV moderate-low — acceptable"
        elif iv_rank < 60:
            signal = "🟠 IV mid-range — neutral"
        elif iv_rank < 80:
            signal = "🔴 IV elevated — expensive"
        else:
            signal = "🔴🔴 IV HIGH — worst time to buy"

        iv_rank_rows.append(
            {
                "Ticker": ticker,
                "Current HV30": current_hv,
                "HV 52W Low": hv_52w_low,
                "HV 52W High": hv_52w_high,
                "ATM IV": current_atm_iv,
                "IV Rank": iv_rank,
                "IV Percentile": iv_percentile,
                "IV/HV Premium": iv_hv_premium,
                "Signal": signal,
            }
        )
        print(
            f"  {ticker:5s}: IV Rank={iv_rank:.0f}  IV%ile={iv_percentile:.0f}  HV30={current_hv:.1%}  → {signal}"
        )
    except Exception as e:
        print(f"  ⚠ {ticker}: {e}")

iv_rank_df = pd.DataFrame(iv_rank_rows)

if not iv_rank_df.empty:
    display(
        iv_rank_df.style.format(
            {
                "Current HV30": "{:.1%}",
                "HV 52W Low": "{:.1%}",
                "HV 52W High": "{:.1%}",
                "ATM IV": "{:.1%}",
                "IV Rank": "{:.0f}",
                "IV Percentile": "{:.0f}",
                "IV/HV Premium": "{:+.0f}%",
            },
            na_rep="—",
        )
        .background_gradient(subset=["IV Rank"], cmap="RdYlGn_r", vmin=0, vmax=100)
        .set_caption("IV Rank: 0 = cheapest, 100 = most expensive")
    )

    # HV history chart
    display(Markdown("### 📈 Volatility History"))
    fig_ivr = make_subplots(
        rows=len(CONVICTION_TICKERS),
        cols=1,
        subplot_titles=[f"{t} — Rolling 30-Day HV" for t in CONVICTION_TICKERS],
        vertical_spacing=0.08,
    )
    for i, ticker in enumerate(CONVICTION_TICKERS, 1):
        try:
            t = yf.Ticker(ticker)
            time.sleep(RATE_LIMIT_SLEEP)
            hist = t.history(period="1y")
            if hist.empty:
                continue
            closes = hist["Close"].dropna()
            log_ret = np.log(closes / closes.shift(1)).dropna()
            rhv = (log_ret.rolling(30).std() * math.sqrt(252)).dropna()
            fig_ivr.add_trace(
                go.Scatter(
                    x=rhv.index,
                    y=rhv.values,
                    mode="lines",
                    name=f"{ticker} HV30",
                    line=dict(width=2),
                ),
                row=i,
                col=1,
            )
            fig_ivr.add_hline(
                y=float(rhv.iloc[-1]),
                row=i,
                col=1,
                line_dash="dash",
                line_color="red",
                annotation_text=f"Now: {float(rhv.iloc[-1]):.1%}",
            )
            fig_ivr.add_hline(
                y=float(rhv.min()),
                row=i,
                col=1,
                line_dash="dot",
                line_color="green",
                annotation_text=f"Low: {float(rhv.min()):.1%}",
            )
            fig_ivr.add_hline(
                y=float(rhv.max()),
                row=i,
                col=1,
                line_dash="dot",
                line_color="red",
                annotation_text=f"High: {float(rhv.max()):.1%}",
            )
            fig_ivr.update_yaxes(tickformat=".0%", row=i, col=1)
        except Exception:
            pass
    fig_ivr.update_layout(
        height=300 * len(CONVICTION_TICKERS),
        title_text="Where Are We in the Vol Cycle?",
        showlegend=False,
    )
    fig_ivr.show()


# ══════════════════════════════════════════════════════════════════
#  💰  SPREAD SLIPPAGE
# ══════════════════════════════════════════════════════════════════
display(Markdown("---\n## 💰 Spread Slippage — Real Fill Cost"))

slippage_rows = []
for _, row in conc_df.iterrows():
    ticker, mid, contracts = row["Ticker"], row["Mid"], row["Contracts"]
    match = focus_chain_df[
        (focus_chain_df["ticker"] == ticker)
        & (abs(focus_chain_df["strike"] - row["Strike"]) < 0.01)
        & (abs(focus_chain_df["dte"] - row["DTE"]) < 3)
    ]
    if not match.empty:
        bid, ask = float(match.iloc[0]["bid"]), float(match.iloc[0]["ask"])
    else:
        sp = focus_scored_df[focus_scored_df["Ticker"] == ticker]["Spread %"].median()
        sp = sp if not np.isnan(sp) else 0.05
        bid, ask = mid * (1 - sp / 2), mid * (1 + sp / 2)

    spread = ask - bid
    spread_pct = spread / mid * 100 if mid > 0 else 0
    entry_slip = (ask - mid) * contracts * 100
    exit_slip = (mid - bid) * contracts * 100
    round_trip = entry_slip + exit_slip
    round_trip_pct = round_trip / (mid * contracts * 100) * 100 if mid > 0 else 0

    slippage_rows.append(
        {
            "Ticker": ticker,
            "Strike": row["Strike"],
            "Mid": mid,
            "Bid": bid,
            "Ask": ask,
            "Spread $": spread,
            "Spread %": spread_pct,
            "Contracts": contracts,
            "Entry Slip $": entry_slip,
            "Exit Slip $": exit_slip,
            "Round-Trip $": round_trip,
            "Round-Trip %": round_trip_pct,
        }
    )

slippage_df = pd.DataFrame(slippage_rows)
total_slippage = slippage_df["Round-Trip $"].sum()

display(
    slippage_df.style.format(
        {
            "Strike": "${:,.0f}",
            "Mid": "${:,.2f}",
            "Bid": "${:,.2f}",
            "Ask": "${:,.2f}",
            "Spread $": "${:,.2f}",
            "Spread %": "{:.1f}%",
            "Entry Slip $": "${:,.0f}",
            "Exit Slip $": "${:,.0f}",
            "Round-Trip $": "${:,.0f}",
            "Round-Trip %": "{:.1f}%",
        }
    )
    .background_gradient(subset=["Round-Trip %"], cmap="Reds", vmin=0, vmax=10)
    .set_caption("Bid-Ask Slippage — your hidden cost")
)

display(
    Markdown(f"""
### 💸 Slippage Impact
| | |
|---|---|
| **Total round-trip slippage** | **${total_slippage:,.0f}** |
| **As % of portfolio** | **{total_slippage / conc_deployed * 100:.1f}%** |
| **Real entry cost (at ask)** | **${conc_deployed + slippage_df["Entry Slip $"].sum():,.0f}** vs ${conc_deployed:,.0f} at mid |

> 💡 Use **limit orders at mid**. Never market-order wide-spread options.
""")
)

---
## 📊 IV Rank & IV Percentile — Entry Timing

> **IV Rank** = where current IV sits vs 52-week range (0 = cheapest, 100 = most expensive).
> 🟢 Buy when IV Rank < 30.  🔴 Avoid buying when IV Rank > 70.

  GEV  : IV Rank=25  IV%ile=40  HV30=42.7%  → 🟡 IV moderate-low — acceptable
  WDC  : IV Rank=100  IV%ile=100  HV30=94.8%  → 🔴🔴 IV HIGH — worst time to buy
  KLAC : IV Rank=94  IV%ile=98  HV30=73.7%  → 🔴🔴 IV HIGH — worst time to buy


,Ticker,Current HV30,HV 52W Low,HV 52W High,ATM IV,IV Rank,IV Percentile,IV/HV Premium,Signal
0,GEV,42.7%,28.6%,84.7%,52.1%,25,40,+22%,🟡 IV moderate-low — acceptable
1,WDC,94.8%,21.1%,94.8%,86.0%,100,100,-9%,🔴🔴 IV HIGH — worst time to buy
2,KLAC,73.7%,24.0%,76.8%,51.6%,94,98,-30%,🔴🔴 IV HIGH — worst time to buy


### 📈 Volatility History

---
## 💰 Spread Slippage — Real Fill Cost

,Ticker,Strike,Mid,Bid,Ask,Spread $,Spread %,Contracts,Entry Slip $,Exit Slip $,Round-Trip $,Round-Trip %
0,GEV,$785,$45.55,$43.60,$47.50,$3.90,8.6%,1,$195,$195,$390,8.6%
1,WDC,$250,$47.62,$45.50,$49.75,$4.25,8.9%,1,$212,$212,$425,8.9%
2,KLAC,"$1,580",$46.10,$42.80,$49.40,$6.60,14.3%,1,$330,$330,$660,14.3%



### 💸 Slippage Impact
| | |
|---|---|
| **Total round-trip slippage** | **$1,475** |
| **As % of portfolio** | **10.6%** |
| **Real entry cost (at ask)** | **$14,665** vs $13,928 at mid |

> 💡 Use **limit orders at mid**. Never market-order wide-spread options.


In [ ]:
# ══════════════════════════════════════════════════════════════════
#  📅  STEP 6: Earnings + Exit Rules + Stress Test + Pre-Trade Checklist
# ══════════════════════════════════════════════════════════════════

display(Markdown("---\n## 📅 Earnings Calendar — IV Crush Risk"))

earnings_rows = []
for ticker in CONVICTION_TICKERS:
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        earnings_dates = []
        try:
            cal = t.calendar
            if isinstance(cal, dict) and "Earnings Date" in cal:
                ed = cal["Earnings Date"]
                if isinstance(ed, list):
                    earnings_dates.extend(
                        [d for d in ed if isinstance(d, (date, datetime))]
                    )
                elif isinstance(ed, (date, datetime)):
                    earnings_dates.append(ed)
        except Exception:
            pass
        try:
            ed_df = t.earnings_dates
            if ed_df is not None and not ed_df.empty:
                for idx in ed_df.index:
                    if hasattr(idx, "date"):
                        earnings_dates.append(
                            idx.date() if hasattr(idx, "date") else idx
                        )
        except Exception:
            pass

        future_earnings = sorted(
            set(
                [
                    d.date() if isinstance(d, datetime) else d
                    for d in earnings_dates
                    if (d.date() if isinstance(d, datetime) else d) >= today
                ]
            )
        )

        for _, pos in conc_df[conc_df["Ticker"] == ticker].iterrows():
            exp_date = (
                datetime.strptime(pos["Expiry"], "%Y-%m-%d").date()
                if isinstance(pos["Expiry"], str)
                else pos["Expiry"]
            )
            dte = pos["DTE"]
            earnings_before_exp = [d for d in future_earnings if d < exp_date]
            next_earnings = future_earnings[0] if future_earnings else None
            days_to_earnings = (next_earnings - today).days if next_earnings else np.nan

            if earnings_before_exp:
                if dte <= 30:
                    risk = "🔴 HIGH — IV crush risk severe"
                elif dte <= 90:
                    risk = "🟡 MODERATE — plan for IV crush"
                else:
                    risk = "🟢 LOW — LEAPS can absorb crush"
            else:
                risk = "✅ CLEAR — no earnings before expiry"

            earnings_rows.append(
                {
                    "Ticker": ticker,
                    "Position": f"${pos['Strike']:,.0f} {pos['Expiry']}",
                    "DTE": dte,
                    "Next Earnings": next_earnings.strftime("%Y-%m-%d")
                    if next_earnings
                    else "Unknown",
                    "Days to Earnings": days_to_earnings,
                    "Earnings Before Expiry": len(earnings_before_exp),
                    "Risk": risk,
                }
            )
    except Exception as e:
        print(f"  ⚠ {ticker}: {e}")
        for _, pos in conc_df[conc_df["Ticker"] == ticker].iterrows():
            earnings_rows.append(
                {
                    "Ticker": ticker,
                    "Position": f"${pos['Strike']:,.0f} {pos['Expiry']}",
                    "DTE": pos["DTE"],
                    "Next Earnings": "Unknown",
                    "Days to Earnings": np.nan,
                    "Earnings Before Expiry": "?",
                    "Risk": "⚠️ UNKNOWN",
                }
            )

earnings_df = pd.DataFrame(earnings_rows)
if not earnings_df.empty:
    display(
        earnings_df.style.format(
            {"Days to Earnings": "{:.0f}"}, na_rep="—"
        ).set_caption("Earnings Calendar vs Positions")
    )


# ══════════════════════════════════════════════════════════════════
#  🎯  EXIT RULES
# ══════════════════════════════════════════════════════════════════
display(Markdown("---\n## 🎯 Exit Rules — Your Playbook"))

exit_rules = []
for _, row in conc_df.iterrows():
    dte, cost = row["DTE"], row["Total Cost"]
    if dte >= 300:
        profit_target, stop_loss, roll_dte, close_dte, trail_stop = 100, 40, 90, 60, 30
    elif dte >= 30:
        profit_target, stop_loss, roll_dte, close_dte, trail_stop = 50, 30, 14, 7, 25
    else:
        profit_target, stop_loss, roll_dte, close_dte, trail_stop = 30, 25, 5, 2, 20

    profit_price = cost * (1 + profit_target / 100)
    stop_price = cost * (1 - stop_loss / 100)
    be_stock = row["Strike"] + row["Mid"]
    needed_intrinsic = profit_price / (row["Contracts"] * 100)
    target_stock = row["Strike"] + needed_intrinsic

    exit_rules.append(
        {
            "Ticker": row["Ticker"],
            "Bucket": row["Bucket"],
            "Strike": row["Strike"],
            "DTE": dte,
            "Entry Cost": cost,
            "🎯 Profit Target": f"+{profit_target}% (${profit_price:,.0f})",
            "🛑 Stop Loss": f"−{stop_loss}% (${stop_price:,.0f})",
            "📉 Trailing Stop": f"{trail_stop}% from peak",
            "🔄 Roll at DTE": f"{roll_dte} DTE",
            "⏹️ Hard Close": f"{close_dte} DTE",
            "BE Stock Price": be_stock,
            "Target Stock": target_stock,
        }
    )

exit_df = pd.DataFrame(exit_rules)
display(
    exit_df.style.format(
        {
            "Strike": "${:,.0f}",
            "Entry Cost": "${:,.0f}",
            "BE Stock Price": "${:,.2f}",
            "Target Stock": "${:,.2f}",
        }
    ).set_caption("Exit playbook — print this")
)

# Price target zones chart
display(Markdown("### 📊 Price Target Zones"))
fig_exit = make_subplots(
    rows=1, cols=len(CONVICTION_TICKERS), subplot_titles=CONVICTION_TICKERS
)
for i, ticker in enumerate(CONVICTION_TICKERS, 1):
    pos = conc_df[conc_df["Ticker"] == ticker]
    if pos.empty:
        continue
    pos = pos.iloc[0]
    spot = spot_map.get(ticker, pos["Strike"])
    be = pos["Strike"] + pos["Mid"]
    labels = ["−20%", "−10%", "Current", "Break-even", "+10%", "+20%", "+50%"]
    prices = [spot * 0.8, spot * 0.9, spot, be, spot * 1.1, spot * 1.2, spot * 1.5]
    colors = [
        "#e74c3c",
        "#e67e22",
        "#3498db",
        "#f39c12",
        "#27ae60",
        "#2ecc71",
        "#1abc9c",
    ]
    fig_exit.add_trace(
        go.Bar(
            x=labels,
            y=prices,
            marker_color=colors,
            text=[f"${p:,.0f}" for p in prices],
            textposition="outside",
            showlegend=False,
        ),
        row=1,
        col=i,
    )
    fig_exit.add_hline(
        y=be,
        row=1,
        col=i,
        line_dash="dash",
        line_color="orange",
        annotation_text=f"BE: ${be:,.0f}",
    )
fig_exit.update_layout(height=450, title_text="Stock Price Zones")
fig_exit.show()


# ══════════════════════════════════════════════════════════════════
#  💀  STRESS TEST + SURVIVAL
# ══════════════════════════════════════════════════════════════════
display(Markdown("---\n## 💀 Stress Test & Portfolio Survival"))

max_loss = conc_deployed
max_loss_pct = max_loss / STARTING_BALANCE * 100

corr_scenarios = [
    ("All flat (theta only, 30 days)", 0.00, 30),
    ("All down 5%", -0.05, 0),
    ("All down 10%", -0.10, 0),
    ("All down 15%", -0.15, 0),
    ("All down 20%", -0.20, 0),
    ("Sector crash −30%", -0.30, 0),
    ("Black swan −50%", -0.50, 0),
]

surv_rows = []
for label, move, theta_days in corr_scenarios:
    total_pnl = 0
    for _, row in conc_df.iterrows():
        spot = spot_map.get(row["Ticker"], row["Strike"])
        new_spot = spot * (1 + move)
        contracts, cost = row["Contracts"], row["Total Cost"]
        if theta_days > 0:
            K, sigma, r = row["Strike"], row.get("IV", 0.30), RISK_FREE_RATE
            T_now = row["DTE"] / 365.0
            T_future = max((row["DTE"] - theta_days) / 365.0, 0.001)
            val_future = bsm_call_price(new_spot, K, T_future, r, sigma) or 0
            pnl = (val_future - row["Mid"]) * contracts * 100
        else:
            intrinsic = max(0, new_spot - row["Strike"]) * contracts * 100
            pnl = intrinsic - cost
        total_pnl += pnl
    port_value = STARTING_BALANCE + total_pnl
    surv_rows.append(
        {
            "Scenario": label,
            "Portfolio P&L": total_pnl,
            "Port Value": port_value,
            "Drawdown %": total_pnl / STARTING_BALANCE * 100,
            "Survival": "✅"
            if port_value > STARTING_BALANCE * 0.50
            else "⚠️"
            if port_value > STARTING_BALANCE * 0.25
            else "💀",
        }
    )

surv_df = pd.DataFrame(surv_rows)
display(
    surv_df.style.format(
        {
            "Portfolio P&L": "${:+,.0f}",
            "Port Value": "${:,.0f}",
            "Drawdown %": "{:+.1f}%",
        }
    )
    .background_gradient(subset=["Drawdown %"], cmap="RdYlGn", vmin=-100, vmax=10)
    .set_caption("Correlated Drawdown Stress Test")
)


# ── Risk of Ruin ──
display(Markdown("### 🎲 Risk of Ruin"))
try:
    joint_returns = pd.DataFrame()
    for ticker in CONVICTION_TICKERS:
        try:
            t = yf.Ticker(ticker)
            time.sleep(RATE_LIMIT_SLEEP)
            h = t.history(period="2y")
            if not h.empty and len(h) > 60:
                joint_returns[ticker] = h["Close"].pct_change().dropna()
        except Exception:
            pass

    if len(joint_returns.columns) > 1:
        joint_returns = joint_returns.dropna()
        monthly = joint_returns.resample("ME").apply(lambda x: (1 + x).prod() - 1)
        all_down = (monthly < 0).all(axis=1)
        prob_all_down = all_down.sum() / len(monthly) * 100 if len(monthly) > 0 else 0
        all_down_10 = (monthly < -0.10).all(axis=1)
        prob_all_down_10 = (
            all_down_10.sum() / len(monthly) * 100 if len(monthly) > 0 else 0
        )
        cols_for_avg = [t for t in CONVICTION_TICKERS if t in monthly.columns]
        monthly["Portfolio"] = (
            monthly[cols_for_avg].mean(axis=1) if cols_for_avg else monthly.mean(axis=1)
        )
        worst_month = monthly["Portfolio"].min()
        worst_month_date = monthly["Portfolio"].idxmin()

        display(
            Markdown(f"""
| Risk Metric | Value |
|-------------|-------|
| **Max possible loss** | ${max_loss:,.0f} ({max_loss_pct:.0f}% of capital) |
| **P(all down same month)** | {prob_all_down:.0f}% |
| **P(all down >10% same month)** | {prob_all_down_10:.0f}% |
| **Worst joint month** | {worst_month:+.1%} ({worst_month_date.strftime("%Y-%m") if hasattr(worst_month_date, "strftime") else worst_month_date}) |
| **Cash reserve** | ${conc_cash:,.0f} ({100 - conc_pct:.0f}%) |
""")
        )
except Exception as e:
    display(Markdown(f"⚠️ Could not compute joint risk: {e}"))


# ══════════════════════════════════════════════════════════════════
#  ✅  PRE-TRADE CHECKLIST
# ══════════════════════════════════════════════════════════════════
display(Markdown("---\n## ✅ Pre-Trade Checklist"))

checklist = []
checklist.append(
    (
        "📐 Greeks computed",
        "✅",
        f"Δ${total_delta_dollars:+,.0f}, Θ${total_theta_daily:,.2f}/day, V${total_vega_dollars:+,.0f}",
    )
)

theta_ok = total_theta_abs < conc_deployed * 0.003
checklist.append(
    (
        "⏰ Theta manageable (<0.3%/day)",
        "✅" if theta_ok else "⚠️",
        f"Θ = {total_theta_abs / conc_deployed * 100:.3f}%/day = ${total_theta_abs * 30:,.0f}/month",
    )
)

if not iv_rank_df.empty:
    avg_rank = iv_rank_df["IV Rank"].mean()
    iv_ok = avg_rank < 50
    checklist.append(
        (
            "📊 IV Rank < 50 (not overpaying)",
            "✅" if iv_ok else "🔴",
            f"Avg IV Rank = {avg_rank:.0f}",
        )
    )

if not earnings_df.empty:
    has_earnings_risk = any("HIGH" in str(r) for r in earnings_df["Risk"])
    checklist.append(
        (
            "📅 No earnings IV crush risk",
            "✅" if not has_earnings_risk else "🔴",
            f"{len(earnings_df)} positions checked",
        )
    )

spread_ok = total_slippage < conc_deployed * 0.05
checklist.append(
    (
        "💰 Spread slippage < 5%",
        "✅" if spread_ok else "⚠️",
        f"Round-trip: ${total_slippage:,.0f} ({total_slippage / conc_deployed * 100:.1f}%)",
    )
)

max_pos_pct = (
    (conc_df["Total Cost"].max() / STARTING_BALANCE * 100) if not conc_df.empty else 0
)
checklist.append(
    (
        "📐 No single position > 35%",
        "✅" if max_pos_pct < 35 else "⚠️",
        f"Largest = {max_pos_pct:.0f}%",
    )
)

checklist.append(
    (
        "💀 Max loss survivable",
        "✅" if max_loss_pct < 95 else "🔴",
        f"Max loss = ${max_loss:,.0f} ({max_loss_pct:.0f}%)",
    )
)

checklist.append(
    ("🎯 Exit rules defined", "✅", "Profit target, stop, trail, roll DTE all set")
)

min_oi = (
    conc_df.merge(
        focus_scored_df[["Ticker", "Strike", "DTE", "OI"]].drop_duplicates(),
        on=["Ticker", "Strike", "DTE"],
        how="left",
    )["OI"].min()
    if not conc_df.empty
    else 0
)
liq_ok = min_oi >= 50 if not np.isnan(min_oi) else False
checklist.append(
    (
        "📈 Liquidity (OI ≥ 50)",
        "✅" if liq_ok else "⚠️",
        f"Min OI = {min_oi:.0f}" if not np.isnan(min_oi) else "Could not verify",
    )
)

check_df = pd.DataFrame(checklist, columns=["Check", "Status", "Detail"])
pass_count = sum(1 for _, s, _ in checklist if s == "✅")
total_checks = len(checklist)

display(
    Markdown(
        f"### {'🟢' if pass_count == total_checks else '🟡' if pass_count >= total_checks - 2 else '🔴'} {pass_count}/{total_checks} checks passed"
    )
)
display(check_df.style.set_caption("Pre-trade readiness checklist"))

if pass_count == total_checks:
    display(Markdown("### 🟢 ALL CLEAR — Ready to execute"))
elif pass_count >= total_checks - 2:
    display(Markdown("### 🟡 MOSTLY CLEAR — Review flagged items"))
else:
    display(Markdown("### 🔴 CAUTION — Multiple flags need attention"))

display(
    Markdown(
        "> **Order of execution:** Place limit orders at mid. Start with highest-conviction position. "
        "Wait for fills before sizing the next. Walk limit up in $0.05 increments if needed — never market-order."
    )
)

---
## 📅 Earnings Calendar — IV Crush Risk

,Ticker,Position,DTE,Next Earnings,Days to Earnings,Earnings Before Expiry,Risk
0,GEV,$785 2026-03-13,33,2026-04-29,80,0,✅ CLEAR — no earnings before expiry
1,WDC,$250 2026-03-13,33,Unknown,—,0,✅ CLEAR — no earnings before expiry
2,KLAC,"$1,580 2026-03-20",40,2026-04-29,80,0,✅ CLEAR — no earnings before expiry


---
## 🎯 Exit Rules — Your Playbook

,Ticker,Bucket,Strike,DTE,Entry Cost,🎯 Profit Target,🛑 Stop Loss,📉 Trailing Stop,🔄 Roll at DTE,⏹️ Hard Close,BE Stock Price,Target Stock
0,GEV,🔄 30-90 DTE,$785,33,"$4,555","+50% ($6,832)","−30% ($3,188)",25% from peak,14 DTE,7 DTE,$830.55,$853.33
1,WDC,🔄 30-90 DTE,$250,33,"$4,762","+50% ($7,144)","−30% ($3,334)",25% from peak,14 DTE,7 DTE,$297.62,$321.44
2,KLAC,🔄 30-90 DTE,"$1,580",40,"$4,610","+50% ($6,915)","−30% ($3,227)",25% from peak,14 DTE,7 DTE,"$1,626.10","$1,649.15"


### 📊 Price Target Zones

...

### 🔴 5/9 checks passed

,Check,Status,Detail
0,📐 Greeks computed,✅,"Δ$+110,064, Θ$-233.50/day, V$+296"
1,⏰ Theta manageable (<0.3%/day),⚠️,"Θ = 1.677%/day = $7,005/month"
2,📊 IV Rank < 50 (not overpaying),🔴,Avg IV Rank = 73
3,📅 No earnings IV crush risk,✅,3 positions checked
4,💰 Spread slippage < 5%,⚠️,"Round-trip: $1,475 (10.6%)"
5,📐 No single position > 35%,✅,Largest = 32%
6,💀 Max loss survivable,✅,"Max loss = $13,928 (93%)"
7,🎯 Exit rules defined,✅,"Profit target, stop, trail, roll DTE all set"
8,📈 Liquidity (OI ≥ 50),⚠️,Min OI = 1


### 🔴 CAUTION — Multiple flags need attention

> **Order of execution:** Place limit orders at mid. Start with highest-conviction position. Wait for fills before sizing the next. Walk limit up in $0.05 increments if needed — never market-order.

: 